In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.6.0'

In [3]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels


Using TensorFlow backend.


In [4]:
ROOT_PATH = '/home/tang/targa/BigSize1/'
train_data_directory = os.path.join(ROOT_PATH, "train")
validation_data_directory = os.path.join(ROOT_PATH, "val")
test_data_directory = os.path.join(ROOT_PATH, "test")
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)
validation_images,validation_labels = load_data(validation_data_directory)

In [5]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL,valL=array(train_labels),array(test_labels),array(validation_labels)
label_encoder = LabelEncoder()

In [6]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)
integer_encoded_valL = label_encoder.fit_transform(valL)

In [7]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)
validationLabels = np_utils.to_categorical(integer_encoded_valL,num_classes = 32)

In [8]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))
valImages = rgb2gray(np.array(validation_images))
train_flat_images = np.reshape(trainImages,(len(trainImages),160*90))
test_flat_images = np.reshape(testImages,(len(testImages),160*90))
val_flat_images = np.reshape(valImages,(len(valImages),160*90))

In [9]:
valImages.shape

(37312, 160, 90)

In [10]:
train_flat_images[0].shape

(14400,)

In [11]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))

print('val images shape:\t\t{}'.format(val_flat_images.shape))

train images shape:		(37312, 14400)
test images shape:		(37312, 14400)
val images shape:		(37312, 14400)


In [12]:
integer_encoded_trL[0]

18

In [13]:
validationLabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
print('Size of:')
print('- Training-set:\t\t{}'.format(len(trainLabels)))
print('- Test-set:\t\t{}'.format(len(testLabels)))
print('- Validation-set:\t{}'.format(len(validation_labels)))

Size of:
- Training-set:		37312
- Test-set:		37312
- Validation-set:	37312


In [15]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.1,dtype = tf.float32)

In [20]:
#命名空间 
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,14400],name = 'x-input')
    y = tf.placeholder(tf.float32,[None,32],name = 'y-input')
#定义一个dropout的值
keep_prob = tf.placeholder(tf.float32)
#第一层
W1 = tf.Variable(tf.truncated_normal([14400,1000],stddev=0.1))#改成截断正太分布
b1 = tf.Variable(tf.zeros([1000])+0.1)
output1 = tf.matmul(x,W1) + b1
L1 = tf.nn.tanh(output1)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L1_drop = tf.nn.dropout(L1,keep_prob)

#第二层
W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.2)
output2 = tf.matmul(L1_drop,W2)+b2
L2 = tf.nn.tanh(output2)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L2_drop = tf.nn.dropout(L2,keep_prob)



#第三层
W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
output3 = tf.matmul(L2_drop,W3)+b3
L3 = tf.nn.tanh(output3)
#L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
L3_drop = tf.nn.dropout(L3,keep_prob)

# 第四层
W4 = tf.Variable(tf.truncated_normal([1000,32],stddev=0.1))
b4 = tf.Variable(tf.zeros([32])+0.1)
output4 = tf.matmul(L3_drop,W4)+ b4


#预测分类
prediction = tf.nn.softmax(output4)

#定义一个损失函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
#loss = tf.reduce_mean(tf.square(y-prediction))
#定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
train_step = tf.train.AdagradOptimizer(lr).minimize(loss)
#train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()

#定义一个求准确率的方法
#如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值

#求准确率
#首先把bool值转化成32位的浮点数，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [21]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    write = tf.summary.FileWriter('logs/targaFourLayerChangeLr')
    for epoch in range(1000):#把所有的图片训练20次
        #设置学习率初始为０．００１在ａｄａｍ optimizer，然后随着ｅｐｏｃｈ指数降低
        sess.run(tf.assign(lr,0.1*(1.0**epoch)))
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        
        acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        learning_rate = sess.run(lr)
        print('epoch=' + str(epoch) + ',test accuracy=' + str(acc)+',train accuracy='+ str(train_acc)+',learning rate='+str(learning_rate))

epoch=0,test accuracy=0.48558104,train accuracy=0.50203687,learning rate=0.1
epoch=1,test accuracy=0.5385667,train accuracy=0.5593107,learning rate=0.1
epoch=2,test accuracy=0.54751825,train accuracy=0.5722288,learning rate=0.1
epoch=3,test accuracy=0.55561215,train accuracy=0.5851201,learning rate=0.1
epoch=4,test accuracy=0.5580242,train accuracy=0.59511685,learning rate=0.1
epoch=5,test accuracy=0.5712103,train accuracy=0.60873175,learning rate=0.1
epoch=6,test accuracy=0.53784305,train accuracy=0.57587373,learning rate=0.1
epoch=7,test accuracy=0.56652015,train accuracy=0.6096698,learning rate=0.1
epoch=8,test accuracy=0.5754985,train accuracy=0.6223735,learning rate=0.1
epoch=9,test accuracy=0.5614548,train accuracy=0.6075257,learning rate=0.1
epoch=10,test accuracy=0.58037627,train accuracy=0.63239706,learning rate=0.1
epoch=11,test accuracy=0.587559,train accuracy=0.64156306,learning rate=0.1
epoch=12,test accuracy=0.57927746,train accuracy=0.63454115,learning rate=0.1
epoch=13,

epoch=107,test accuracy=0.70955724,train accuracy=0.8250161,learning rate=0.1
epoch=108,test accuracy=0.7097449,train accuracy=0.8259809,learning rate=0.1
epoch=109,test accuracy=0.702991,train accuracy=0.8224432,learning rate=0.1
epoch=110,test accuracy=0.70475984,train accuracy=0.82461405,learning rate=0.1
epoch=111,test accuracy=0.70111495,train accuracy=0.8204599,learning rate=0.1
epoch=112,test accuracy=0.7020798,train accuracy=0.8228988,learning rate=0.1
epoch=113,test accuracy=0.70663595,train accuracy=0.82611495,learning rate=0.1
epoch=114,test accuracy=0.70730597,train accuracy=0.82764256,learning rate=0.1
epoch=115,test accuracy=0.707172,train accuracy=0.8262221,learning rate=0.1
epoch=116,test accuracy=0.70912844,train accuracy=0.827589,learning rate=0.1
epoch=117,test accuracy=0.7053227,train accuracy=0.82662416,learning rate=0.1
epoch=118,test accuracy=0.707842,train accuracy=0.82986706,learning rate=0.1
epoch=119,test accuracy=0.7063143,train accuracy=0.8282054,learning r

epoch=213,test accuracy=0.7206797,train accuracy=0.85843694,learning rate=0.1
epoch=214,test accuracy=0.72614706,train accuracy=0.86141187,learning rate=0.1
epoch=215,test accuracy=0.72445863,train accuracy=0.8609295,learning rate=0.1
epoch=216,test accuracy=0.7265223,train accuracy=0.8622963,learning rate=0.1
epoch=217,test accuracy=0.7262007,train accuracy=0.8618139,learning rate=0.1
epoch=218,test accuracy=0.7256647,train accuracy=0.8619479,learning rate=0.1
epoch=219,test accuracy=0.7235206,train accuracy=0.86058104,learning rate=0.1
epoch=220,test accuracy=0.7267635,train accuracy=0.8632075,learning rate=0.1
epoch=221,test accuracy=0.7254503,train accuracy=0.8621891,learning rate=0.1
epoch=222,test accuracy=0.7266831,train accuracy=0.86347556,learning rate=0.1
epoch=223,test accuracy=0.7262007,train accuracy=0.86293954,learning rate=0.1
epoch=224,test accuracy=0.72290415,train accuracy=0.86154586,learning rate=0.1
epoch=225,test accuracy=0.7293364,train accuracy=0.8640652,learning

epoch=319,test accuracy=0.73212373,train accuracy=0.87438357,learning rate=0.1
epoch=320,test accuracy=0.7329009,train accuracy=0.87548244,learning rate=0.1
epoch=321,test accuracy=0.73086405,train accuracy=0.874464,learning rate=0.1
epoch=322,test accuracy=0.72997963,train accuracy=0.87328476,learning rate=0.1
epoch=323,test accuracy=0.7312125,train accuracy=0.87355274,learning rate=0.1
epoch=324,test accuracy=0.73056924,train accuracy=0.87392795,learning rate=0.1
epoch=325,test accuracy=0.73236495,train accuracy=0.8752948,learning rate=0.1
epoch=326,test accuracy=0.7298188,train accuracy=0.8744908,learning rate=0.1
epoch=327,test accuracy=0.73064965,train accuracy=0.874866,learning rate=0.1
epoch=328,test accuracy=0.7320701,train accuracy=0.8752948,learning rate=0.1
epoch=329,test accuracy=0.7294972,train accuracy=0.8751608,learning rate=0.1
epoch=330,test accuracy=0.7293096,train accuracy=0.874464,learning rate=0.1
epoch=331,test accuracy=0.7268439,train accuracy=0.8741692,learning 

epoch=425,test accuracy=0.7294436,train accuracy=0.87829655,learning rate=0.1
epoch=426,test accuracy=0.7302476,train accuracy=0.87851095,learning rate=0.1
epoch=427,test accuracy=0.7295776,train accuracy=0.8776533,learning rate=0.1
epoch=428,test accuracy=0.72807676,train accuracy=0.8775461,learning rate=0.1
epoch=429,test accuracy=0.73389256,train accuracy=0.8795026,learning rate=0.1
epoch=430,test accuracy=0.7346698,train accuracy=0.87931496,learning rate=0.1
epoch=431,test accuracy=0.73249894,train accuracy=0.87936854,learning rate=0.1
epoch=432,test accuracy=0.7283448,train accuracy=0.8775193,learning rate=0.1
epoch=433,test accuracy=0.7309981,train accuracy=0.87867177,learning rate=0.1
epoch=434,test accuracy=0.73048884,train accuracy=0.87848413,learning rate=0.1
epoch=435,test accuracy=0.73324937,train accuracy=0.8792078,learning rate=0.1
epoch=436,test accuracy=0.72947043,train accuracy=0.8785377,learning rate=0.1
epoch=437,test accuracy=0.73094445,train accuracy=0.8783233,lear

epoch=531,test accuracy=0.7348038,train accuracy=0.8834959,learning rate=0.1
epoch=532,test accuracy=0.73785913,train accuracy=0.8847288,learning rate=0.1
epoch=533,test accuracy=0.73936,train accuracy=0.8848896,learning rate=0.1
epoch=534,test accuracy=0.73847556,train accuracy=0.8846484,learning rate=0.1
epoch=535,test accuracy=0.73579544,train accuracy=0.88448757,learning rate=0.1
epoch=536,test accuracy=0.7371891,train accuracy=0.88429993,learning rate=0.1
epoch=537,test accuracy=0.7378323,train accuracy=0.88440716,learning rate=0.1
epoch=538,test accuracy=0.739494,train accuracy=0.884568,learning rate=0.1
epoch=539,test accuracy=0.7376179,train accuracy=0.88491637,learning rate=0.1
epoch=540,test accuracy=0.73550063,train accuracy=0.8848628,learning rate=0.1
epoch=541,test accuracy=0.73324937,train accuracy=0.88389796,learning rate=0.1
epoch=542,test accuracy=0.73274016,train accuracy=0.8838443,learning rate=0.1
epoch=543,test accuracy=0.7375375,train accuracy=0.8849968,learning r

epoch=637,test accuracy=0.738958,train accuracy=0.88668525,learning rate=0.1
epoch=638,test accuracy=0.73844874,train accuracy=0.8869265,learning rate=0.1
epoch=639,test accuracy=0.73826116,train accuracy=0.8870069,learning rate=0.1
epoch=640,test accuracy=0.73638505,train accuracy=0.88684607,learning rate=0.1
epoch=641,test accuracy=0.7344286,train accuracy=0.88668525,learning rate=0.1
epoch=642,test accuracy=0.73855597,train accuracy=0.88684607,learning rate=0.1
epoch=643,test accuracy=0.73684067,train accuracy=0.8870069,learning rate=0.1
epoch=644,test accuracy=0.7388508,train accuracy=0.8870873,learning rate=0.1
epoch=645,test accuracy=0.73743033,train accuracy=0.88695323,learning rate=0.1
epoch=646,test accuracy=0.7367603,train accuracy=0.88695323,learning rate=0.1
epoch=647,test accuracy=0.73743033,train accuracy=0.8871677,learning rate=0.1
epoch=648,test accuracy=0.73617065,train accuracy=0.88652444,learning rate=0.1
epoch=649,test accuracy=0.7364387,train accuracy=0.8872213,lea

epoch=743,test accuracy=0.74581903,train accuracy=0.89464515,learning rate=0.1
epoch=744,test accuracy=0.7424957,train accuracy=0.89485955,learning rate=0.1
epoch=745,test accuracy=0.7432997,train accuracy=0.8943235,learning rate=0.1
epoch=746,test accuracy=0.74413055,train accuracy=0.8944575,learning rate=0.1
epoch=747,test accuracy=0.74383575,train accuracy=0.8941627,learning rate=0.1
epoch=748,test accuracy=0.743943,train accuracy=0.8951008,learning rate=0.1
epoch=749,test accuracy=0.7468375,train accuracy=0.8953152,learning rate=0.1
epoch=750,test accuracy=0.7453098,train accuracy=0.8950472,learning rate=0.1
epoch=751,test accuracy=0.7450686,train accuracy=0.89491314,learning rate=0.1
epoch=752,test accuracy=0.7465963,train accuracy=0.8950472,learning rate=0.1
epoch=753,test accuracy=0.7449346,train accuracy=0.89499354,learning rate=0.1
epoch=754,test accuracy=0.74354094,train accuracy=0.89426994,learning rate=0.1
epoch=755,test accuracy=0.743809,train accuracy=0.89480597,learning 

In [ ]:
#lr = 0.001*(0.95)**epoch
epoch=0test accuracy=0.03125train accuracy=0.03125,learning rate=0.001
epoch=1test accuracy=0.03125train accuracy=0.03125,learning rate=0.00095
epoch=2test accuracy=0.03125train accuracy=0.03125,learning rate=0.0009025
epoch=3test accuracy=0.03125train accuracy=0.03125,learning rate=0.000857375
epoch=4test accuracy=0.03125train accuracy=0.03125,learning rate=0.00081450626
epoch=5test accuracy=0.03125train accuracy=0.03125,learning rate=0.0007737809
epoch=6test accuracy=0.03125train accuracy=0.03125,learning rate=0.0007350919
epoch=7test accuracy=0.03125train accuracy=0.03125,learning rate=0.0006983373
epoch=8test accuracy=0.03125train accuracy=0.03125,learning rate=0.0006634204
epoch=9test accuracy=0.03125train accuracy=0.03125,learning rate=0.0006302494
epoch=10test accuracy=0.03125train accuracy=0.03125,learning rate=0.0005987369
epoch=11test accuracy=0.03125train accuracy=0.03125,learning rate=0.0005688001
epoch=12test accuracy=0.03125train accuracy=0.03125,learning rate=0.0005403601
epoch=13test accuracy=0.03125train accuracy=0.03125,learning rate=0.0005133421
epoch=14test accuracy=0.03125train accuracy=0.03125,learning rate=0.000487675
epoch=15test accuracy=0.03125train accuracy=0.03125,learning rate=0.00046329122
epoch=16test accuracy=0.03125train accuracy=0.03125,learning rate=0.00044012666
epoch=17test accuracy=0.03125train accuracy=0.03125,learning rate=0.00041812033
epoch=18test accuracy=0.03125train accuracy=0.03125,learning rate=0.00039721432
epoch=19test accuracy=0.03125train accuracy=0.03125,learning rate=0.0003773536
epoch=20test accuracy=0.03125train accuracy=0.03125,learning rate=0.00035848594
epoch=21test accuracy=0.03125train accuracy=0.03125,learning rate=0.00034056162
epoch=22test accuracy=0.03125train accuracy=0.03125,learning rate=0.00032353355
epoch=23test accuracy=0.03125train accuracy=0.03125,learning rate=0.00030735688
epoch=24test accuracy=0.03125train accuracy=0.03125,learning rate=0.000291989
epoch=25test accuracy=0.03125train accuracy=0.03125,learning rate=0.00027738957
epoch=26test accuracy=0.03125train accuracy=0.03125,learning rate=0.0002635201
epoch=27test accuracy=0.03125train accuracy=0.03125,learning rate=0.00025034408
epoch=28test accuracy=0.03125train accuracy=0.03125,learning rate=0.00023782688
epoch=29test accuracy=0.03125train accuracy=0.03125,learning rate=0.00022593554
epoch=30test accuracy=0.03125train accuracy=0.03125,learning rate=0.00021463877
epoch=31test accuracy=0.03125train accuracy=0.03125,learning rate=0.00020390682
epoch=32test accuracy=0.03125train accuracy=0.03125,learning rate=0.00019371149
epoch=33test accuracy=0.03125train accuracy=0.03125,learning rate=0.0001840259
epoch=34test accuracy=0.03125train accuracy=0.03125,learning rate=0.00017482461
epoch=35test accuracy=0.03125train accuracy=0.03125,learning rate=0.00016608338
epoch=36test accuracy=0.03125train accuracy=0.03125,learning rate=0.00015777921
epoch=37test accuracy=0.03125train accuracy=0.03125,learning rate=0.00014989026
epoch=38test accuracy=0.03125train accuracy=0.03125,learning rate=0.00014239574
epoch=39test accuracy=0.03125train accuracy=0.03125,learning rate=0.00013527596
epoch=40test accuracy=0.03125train accuracy=0.03125,learning rate=0.00012851215
epoch=41test accuracy=0.03125train accuracy=0.03125,learning rate=0.00012208655
epoch=42test accuracy=0.03125train accuracy=0.03125,learning rate=0.00011598222
epoch=43test accuracy=0.03125train accuracy=0.03125,learning rate=0.00011018311
epoch=44test accuracy=0.03125train accuracy=0.03125,learning rate=0.000104673956
epoch=45test accuracy=0.03125train accuracy=0.03125,learning rate=9.944026e-05
epoch=46test accuracy=0.03125train accuracy=0.03125,learning rate=9.446825e-05
epoch=47test accuracy=0.03125train accuracy=0.03125,learning rate=8.974483e-05
epoch=48test accuracy=0.03125train accuracy=0.03125,learning rate=8.525759e-05
epoch=49test accuracy=0.03125train accuracy=0.03125,learning rate=8.099471e-05
epoch=50test accuracy=0.03125train accuracy=0.03125,learning rate=7.6944976e-05
epoch=51test accuracy=0.03125train accuracy=0.03125,learning rate=7.3097726e-05
epoch=52test accuracy=0.03125train accuracy=0.03125,learning rate=6.944284e-05
epoch=53test accuracy=0.03125train accuracy=0.03125,learning rate=6.59707e-05
epoch=54test accuracy=0.03125train accuracy=0.03125,learning rate=6.267217e-05
epoch=55test accuracy=0.03125train accuracy=0.03125,learning rate=5.9538554e-05
epoch=56test accuracy=0.03125train accuracy=0.03125,learning rate=5.656163e-05
epoch=57test accuracy=0.03125train accuracy=0.03125,learning rate=5.3733547e-05
epoch=58test accuracy=0.03125train accuracy=0.03125,learning rate=5.1046867e-05
epoch=59test accuracy=0.03125train accuracy=0.03125,learning rate=4.8494527e-05
epoch=60test accuracy=0.03125train accuracy=0.03125,learning rate=4.60698e-05
epoch=61test accuracy=0.03125train accuracy=0.03125,learning rate=4.3766307e-05
epoch=62test accuracy=0.03125train accuracy=0.03125,learning rate=4.1577994e-05
epoch=63test accuracy=0.03125train accuracy=0.03125,learning rate=3.9499093e-05
epoch=64test accuracy=0.03125train accuracy=0.03125,learning rate=3.752414e-05
epoch=65test accuracy=0.03125train accuracy=0.03125,learning rate=3.5647932e-05
epoch=66test accuracy=0.03125train accuracy=0.03125,learning rate=3.3865537e-05
epoch=67test accuracy=0.03125train accuracy=0.03125,learning rate=3.217226e-05
epoch=68test accuracy=0.03125train accuracy=0.03125,learning rate=3.0563646e-05
epoch=69test accuracy=0.03125train accuracy=0.03125,learning rate=2.9035464e-05
epoch=70test accuracy=0.03125train accuracy=0.03125,learning rate=2.758369e-05
epoch=71test accuracy=0.03125train accuracy=0.03125,learning rate=2.6204505e-05
epoch=72test accuracy=0.03125train accuracy=0.03125,learning rate=2.4894282e-05
epoch=73test accuracy=0.03125train accuracy=0.03125,learning rate=2.3649567e-05
epoch=74test accuracy=0.03125train accuracy=0.03125,learning rate=2.2467088e-05
epoch=75test accuracy=0.03125train accuracy=0.03125,learning rate=2.1343734e-05
epoch=76test accuracy=0.03125train accuracy=0.03125,learning rate=2.0276548e-05
epoch=77test accuracy=0.03125train accuracy=0.03125,learning rate=1.926272e-05
epoch=78test accuracy=0.03125train accuracy=0.03125,learning rate=1.8299585e-05
epoch=79test accuracy=0.03125train accuracy=0.03125,learning rate=1.7384604e-05
epoch=80test accuracy=0.03125train accuracy=0.03125,learning rate=1.6515374e-05
epoch=81test accuracy=0.03125train accuracy=0.03125,learning rate=1.5689606e-05
epoch=82test accuracy=0.03125train accuracy=0.03125,learning rate=1.4905126e-05
epoch=83test accuracy=0.03125train accuracy=0.03125,learning rate=1.4159869e-05
epoch=84test accuracy=0.03125train accuracy=0.03125,learning rate=1.3451876e-05
epoch=85test accuracy=0.03125train accuracy=0.03125,learning rate=1.2779282e-05
epoch=86test accuracy=0.03125train accuracy=0.03125,learning rate=1.2140318e-05
epoch=87test accuracy=0.03125train accuracy=0.03125,learning rate=1.1533302e-05
epoch=88test accuracy=0.03125train accuracy=0.03125,learning rate=1.0956637e-05
epoch=89test accuracy=0.03125train accuracy=0.03125,learning rate=1.0408805e-05
epoch=90test accuracy=0.03125train accuracy=0.03125,learning rate=9.888365e-06
epoch=91test accuracy=0.03125train accuracy=0.03125,learning rate=9.393946e-06
epoch=92test accuracy=0.03125train accuracy=0.03125,learning rate=8.924249e-06
epoch=93test accuracy=0.03125train accuracy=0.03125,learning rate=8.478037e-06
epoch=94test accuracy=0.03125train accuracy=0.03125,learning rate=8.054135e-06
epoch=95test accuracy=0.03125train accuracy=0.03125,learning rate=7.651428e-06
epoch=96test accuracy=0.03125train accuracy=0.03125,learning rate=7.2688567e-06
epoch=97test accuracy=0.03125train accuracy=0.03125,learning rate=6.905414e-06
epoch=98test accuracy=0.03125train accuracy=0.03125,learning rate=6.560143e-06
epoch=99test accuracy=0.03125train accuracy=0.03125,learning rate=6.232136e-06
epoch=100test accuracy=0.03125train accuracy=0.03125,learning rate=5.920529e-06
epoch=101test accuracy=0.03125train accuracy=0.03125,learning rate=5.6245026e-06
epoch=102test accuracy=0.03125train accuracy=0.03125,learning rate=5.3432777e-06
epoch=103test accuracy=0.03125train accuracy=0.03125,learning rate=5.0761137e-06
epoch=104test accuracy=0.03125train accuracy=0.03125,learning rate=4.8223083e-06
epoch=105test accuracy=0.03125train accuracy=0.03125,learning rate=4.5811926e-06
epoch=106test accuracy=0.03125train accuracy=0.03125,learning rate=4.352133e-06
epoch=107test accuracy=0.03125train accuracy=0.03125,learning rate=4.1345265e-06
epoch=108test accuracy=0.03125train accuracy=0.03125,learning rate=3.9278e-06
epoch=109test accuracy=0.03125train accuracy=0.03125,learning rate=3.73141e-06
epoch=110test accuracy=0.03125train accuracy=0.03125,learning rate=3.5448395e-06
epoch=111test accuracy=0.03125train accuracy=0.03125,learning rate=3.3675976e-06
epoch=112test accuracy=0.03125train accuracy=0.03125,learning rate=3.1992176e-06
epoch=113test accuracy=0.03125train accuracy=0.03125,learning rate=3.0392569e-06
epoch=114test accuracy=0.03125train accuracy=0.03125,learning rate=2.887294e-06
epoch=115test accuracy=0.03125train accuracy=0.03125,learning rate=2.7429292e-06
epoch=116test accuracy=0.03125train accuracy=0.03125,learning rate=2.6057828e-06
epoch=117test accuracy=0.03125train accuracy=0.03125,learning rate=2.4754936e-06
epoch=118test accuracy=0.03125train accuracy=0.03125,learning rate=2.3517189e-06
epoch=119test accuracy=0.03125train accuracy=0.03125,learning rate=2.234133e-06
epoch=120test accuracy=0.03125train accuracy=0.03125,learning rate=2.1224264e-06
epoch=121test accuracy=0.03125train accuracy=0.03125,learning rate=2.016305e-06
epoch=122test accuracy=0.03125train accuracy=0.03125,learning rate=1.9154897e-06
epoch=123test accuracy=0.03125train accuracy=0.03125,learning rate=1.8197153e-06
epoch=124test accuracy=0.03125train accuracy=0.03125,learning rate=1.7287296e-06
epoch=125test accuracy=0.03125train accuracy=0.03125,learning rate=1.6422931e-06
epoch=126test accuracy=0.03125train accuracy=0.03125,learning rate=1.5601785e-06
epoch=127test accuracy=0.03125train accuracy=0.03125,learning rate=1.4821695e-06
epoch=128test accuracy=0.03125train accuracy=0.03125,learning rate=1.408061e-06
epoch=129test accuracy=0.03125train accuracy=0.03125,learning rate=1.337658e-06
epoch=130test accuracy=0.03125train accuracy=0.03125,learning rate=1.2707751e-06
epoch=131test accuracy=0.03125train accuracy=0.03125,learning rate=1.2072363e-06
epoch=132test accuracy=0.03125train accuracy=0.03125,learning rate=1.1468745e-06
epoch=133test accuracy=0.03125train accuracy=0.03125,learning rate=1.0895308e-06
epoch=134test accuracy=0.03125train accuracy=0.03125,learning rate=1.0350542e-06
epoch=135test accuracy=0.03125train accuracy=0.03125,learning rate=9.833016e-07
epoch=136test accuracy=0.03125train accuracy=0.03125,learning rate=9.3413644e-07
epoch=137test accuracy=0.03125train accuracy=0.03125,learning rate=8.874296e-07
epoch=138test accuracy=0.03125train accuracy=0.03125,learning rate=8.4305816e-07
epoch=139test accuracy=0.03125train accuracy=0.03125,learning rate=8.0090524e-07
epoch=140test accuracy=0.03125train accuracy=0.03125,learning rate=7.6085996e-07
epoch=141test accuracy=0.03125train accuracy=0.03125,learning rate=7.22817e-07
epoch=142test accuracy=0.03125train accuracy=0.03125,learning rate=6.866761e-07
epoch=143test accuracy=0.03125train accuracy=0.03125,learning rate=6.5234235e-07
epoch=144test accuracy=0.03125train accuracy=0.03125,learning rate=6.197252e-07
epoch=145test accuracy=0.03125train accuracy=0.03125,learning rate=5.8873894e-07
epoch=146test accuracy=0.03125train accuracy=0.03125,learning rate=5.59302e-07
epoch=147test accuracy=0.03125train accuracy=0.03125,learning rate=5.313369e-07
epoch=148test accuracy=0.03125train accuracy=0.03125,learning rate=5.0477007e-07
epoch=149test accuracy=0.03125train accuracy=0.03125,learning rate=4.7953154e-07
epoch=150test accuracy=0.03125train accuracy=0.03125,learning rate=4.5555498e-07
epoch=151test accuracy=0.03125train accuracy=0.03125,learning rate=4.3277723e-07
epoch=152test accuracy=0.03125train accuracy=0.03125,learning rate=4.1113836e-07
epoch=153test accuracy=0.03125train accuracy=0.03125,learning rate=3.9058145e-07
epoch=154test accuracy=0.03125train accuracy=0.03125,learning rate=3.7105238e-07
epoch=155test accuracy=0.03125train accuracy=0.03125,learning rate=3.5249977e-07
epoch=156test accuracy=0.03125train accuracy=0.03125,learning rate=3.3487476e-07
epoch=157test accuracy=0.03125train accuracy=0.03125,learning rate=3.1813104e-07
epoch=158test accuracy=0.03125train accuracy=0.03125,learning rate=3.0222446e-07
epoch=159test accuracy=0.03125train accuracy=0.03125,learning rate=2.8711327e-07
epoch=160test accuracy=0.03125train accuracy=0.03125,learning rate=2.727576e-07
epoch=161test accuracy=0.03125train accuracy=0.03125,learning rate=2.591197e-07
epoch=162test accuracy=0.03125train accuracy=0.03125,learning rate=2.4616372e-07
epoch=163test accuracy=0.03125train accuracy=0.03125,learning rate=2.3385554e-07
epoch=164test accuracy=0.03125train accuracy=0.03125,learning rate=2.2216277e-07
epoch=165test accuracy=0.03125train accuracy=0.03125,learning rate=2.1105463e-07
epoch=166test accuracy=0.03125train accuracy=0.03125,learning rate=2.0050189e-07
epoch=167test accuracy=0.03125train accuracy=0.03125,learning rate=1.904768e-07
epoch=168test accuracy=0.03125train accuracy=0.03125,learning rate=1.8095295e-07
epoch=169test accuracy=0.03125train accuracy=0.03125,learning rate=1.7190531e-07
epoch=170test accuracy=0.03125train accuracy=0.03125,learning rate=1.6331005e-07
epoch=171test accuracy=0.03125train accuracy=0.03125,learning rate=1.5514455e-07
epoch=172test accuracy=0.03125train accuracy=0.03125,learning rate=1.4738731e-07
epoch=173test accuracy=0.03125train accuracy=0.03125,learning rate=1.4001795e-07
epoch=174test accuracy=0.03125train accuracy=0.03125,learning rate=1.3301705e-07
epoch=175test accuracy=0.03125train accuracy=0.03125,learning rate=1.263662e-07
epoch=176test accuracy=0.03125train accuracy=0.03125,learning rate=1.2004789e-07
epoch=177test accuracy=0.03125train accuracy=0.03125,learning rate=1.140455e-07
epoch=178test accuracy=0.03125train accuracy=0.03125,learning rate=1.0834322e-07
epoch=179test accuracy=0.03125train accuracy=0.03125,learning rate=1.0292606e-07
epoch=180test accuracy=0.03125train accuracy=0.03125,learning rate=9.7779754e-08
epoch=181test accuracy=0.03125train accuracy=0.03125,learning rate=9.2890765e-08
epoch=182test accuracy=0.03125train accuracy=0.03125,learning rate=8.824623e-08
epoch=183test accuracy=0.03125train accuracy=0.03125,learning rate=8.383392e-08
epoch=184test accuracy=0.03125train accuracy=0.03125,learning rate=7.964222e-08
epoch=185test accuracy=0.03125train accuracy=0.03125,learning rate=7.566011e-08
epoch=186test accuracy=0.03125train accuracy=0.03125,learning rate=7.18771e-08
epoch=187test accuracy=0.03125train accuracy=0.03125,learning rate=6.828325e-08
epoch=188test accuracy=0.03125train accuracy=0.03125,learning rate=6.486909e-08
epoch=189test accuracy=0.03125train accuracy=0.03125,learning rate=6.1625634e-08
epoch=190test accuracy=0.03125train accuracy=0.03125,learning rate=5.854435e-08
epoch=191test accuracy=0.03125train accuracy=0.03125,learning rate=5.5617136e-08
epoch=192test accuracy=0.03125train accuracy=0.03125,learning rate=5.2836278e-08
epoch=193test accuracy=0.03125train accuracy=0.03125,learning rate=5.0194465e-08
epoch=194test accuracy=0.03125train accuracy=0.03125,learning rate=4.768474e-08
epoch=195test accuracy=0.03125train accuracy=0.03125,learning rate=4.5300503e-08
epoch=196test accuracy=0.03125train accuracy=0.03125,learning rate=4.3035477e-08
epoch=197test accuracy=0.03125train accuracy=0.03125,learning rate=4.0883705e-08
epoch=198test accuracy=0.03125train accuracy=0.03125,learning rate=3.883952e-08
epoch=199test accuracy=0.03125train accuracy=0.03125,learning rate=3.6897543e-08
epoch=200test accuracy=0.03125train accuracy=0.03125,learning rate=3.5052665e-08
epoch=201test accuracy=0.03125train accuracy=0.03125,learning rate=3.3300033e-08
epoch=202test accuracy=0.03125train accuracy=0.03125,learning rate=3.1635032e-08
epoch=203test accuracy=0.03125train accuracy=0.03125,learning rate=3.005328e-08
epoch=204test accuracy=0.03125train accuracy=0.03125,learning rate=2.8550616e-08
epoch=205test accuracy=0.03125train accuracy=0.03125,learning rate=2.7123084e-08
epoch=206test accuracy=0.03125train accuracy=0.03125,learning rate=2.576693e-08
epoch=207test accuracy=0.03125train accuracy=0.03125,learning rate=2.4478584e-08
epoch=208test accuracy=0.03125train accuracy=0.03125,learning rate=2.3254655e-08
epoch=209test accuracy=0.03125train accuracy=0.03125,learning rate=2.2091923e-08
epoch=210test accuracy=0.03125train accuracy=0.03125,learning rate=2.0987326e-08
epoch=211test accuracy=0.03125train accuracy=0.03125,learning rate=1.993796e-08
epoch=212test accuracy=0.03125train accuracy=0.03125,learning rate=1.8941062e-08
epoch=213test accuracy=0.03125train accuracy=0.03125,learning rate=1.7994008e-08
epoch=214test accuracy=0.03125train accuracy=0.03125,learning rate=1.7094308e-08
epoch=215test accuracy=0.03125train accuracy=0.03125,learning rate=1.6239593e-08
epoch=216test accuracy=0.03125train accuracy=0.03125,learning rate=1.5427613e-08
epoch=217test accuracy=0.03125train accuracy=0.03125,learning rate=1.4656233e-08
epoch=218test accuracy=0.03125train accuracy=0.03125,learning rate=1.3923421e-08
epoch=219test accuracy=0.03125train accuracy=0.03125,learning rate=1.32272495e-08
epoch=220test accuracy=0.03125train accuracy=0.03125,learning rate=1.2565887e-08
epoch=221test accuracy=0.03125train accuracy=0.03125,learning rate=1.1937593e-08
epoch=222test accuracy=0.03125train accuracy=0.03125,learning rate=1.1340713e-08
epoch=223test accuracy=0.03125train accuracy=0.03125,learning rate=1.0773678e-08
epoch=224test accuracy=0.03125train accuracy=0.03125,learning rate=1.0234994e-08
epoch=225test accuracy=0.03125train accuracy=0.03125,learning rate=9.723244e-09
epoch=226test accuracy=0.03125train accuracy=0.03125,learning rate=9.237082e-09
epoch=227test accuracy=0.03125train accuracy=0.03125,learning rate=8.775228e-09
epoch=228test accuracy=0.03125train accuracy=0.03125,learning rate=8.336467e-09
epoch=229test accuracy=0.03125train accuracy=0.03125,learning rate=7.919643e-09
epoch=230test accuracy=0.03125train accuracy=0.03125,learning rate=7.523661e-09
epoch=231test accuracy=0.03125train accuracy=0.03125,learning rate=7.1474777e-09
epoch=232test accuracy=0.03125train accuracy=0.03125,learning rate=6.790104e-09
epoch=233test accuracy=0.03125train accuracy=0.03125,learning rate=6.4505987e-09
epoch=234test accuracy=0.03125train accuracy=0.03125,learning rate=6.1280687e-09
epoch=235test accuracy=0.03125train accuracy=0.03125,learning rate=5.8216654e-09
epoch=236test accuracy=0.03125train accuracy=0.03125,learning rate=5.530582e-09
epoch=237test accuracy=0.03125train accuracy=0.03125,learning rate=5.254053e-09
epoch=238test accuracy=0.03125train accuracy=0.03125,learning rate=4.9913504e-09
epoch=239test accuracy=0.03125train accuracy=0.03125,learning rate=4.741783e-09
epoch=240test accuracy=0.03125train accuracy=0.03125,learning rate=4.504694e-09
epoch=241test accuracy=0.03125train accuracy=0.03125,learning rate=4.279459e-09
epoch=242test accuracy=0.03125train accuracy=0.03125,learning rate=4.065486e-09
epoch=243test accuracy=0.03125train accuracy=0.03125,learning rate=3.862212e-09
epoch=244test accuracy=0.03125train accuracy=0.03125,learning rate=3.6691012e-09
epoch=245test accuracy=0.03125train accuracy=0.03125,learning rate=3.4856462e-09
epoch=246test accuracy=0.03125train accuracy=0.03125,learning rate=3.3113638e-09
epoch=247test accuracy=0.03125train accuracy=0.03125,learning rate=3.1457956e-09
epoch=248test accuracy=0.03125train accuracy=0.03125,learning rate=2.9885059e-09
epoch=249test accuracy=0.03125train accuracy=0.03125,learning rate=2.8390805e-09
epoch=250test accuracy=0.03125train accuracy=0.03125,learning rate=2.6971265e-09
epoch=251test accuracy=0.03125train accuracy=0.03125,learning rate=2.5622702e-09
epoch=252test accuracy=0.03125train accuracy=0.03125,learning rate=2.4341567e-09
epoch=253test accuracy=0.03125train accuracy=0.03125,learning rate=2.312449e-09
epoch=254test accuracy=0.03125train accuracy=0.03125,learning rate=2.1968265e-09
epoch=255test accuracy=0.03125train accuracy=0.03125,learning rate=2.086985e-09
epoch=256test accuracy=0.03125train accuracy=0.03125,learning rate=1.9826358e-09
epoch=257test accuracy=0.03125train accuracy=0.03125,learning rate=1.883504e-09
epoch=258test accuracy=0.03125train accuracy=0.03125,learning rate=1.7893288e-09
epoch=259test accuracy=0.03125train accuracy=0.03125,learning rate=1.6998624e-09
epoch=260test accuracy=0.03125train accuracy=0.03125,learning rate=1.6148692e-09
epoch=261test accuracy=0.03125train accuracy=0.03125,learning rate=1.5341258e-09
epoch=262test accuracy=0.03125train accuracy=0.03125,learning rate=1.4574195e-09
epoch=263test accuracy=0.03125train accuracy=0.03125,learning rate=1.3845486e-09
epoch=264test accuracy=0.03125train accuracy=0.03125,learning rate=1.3153211e-09
epoch=265test accuracy=0.03125train accuracy=0.03125,learning rate=1.2495551e-09
epoch=266test accuracy=0.03125train accuracy=0.03125,learning rate=1.1870773e-09
epoch=267test accuracy=0.03125train accuracy=0.03125,learning rate=1.1277235e-09
epoch=268test accuracy=0.03125train accuracy=0.03125,learning rate=1.0713372e-09
epoch=269test accuracy=0.03125train accuracy=0.03125,learning rate=1.0177704e-09
epoch=270test accuracy=0.03125train accuracy=0.03125,learning rate=9.668819e-10
epoch=271test accuracy=0.03125train accuracy=0.03125,learning rate=9.185378e-10
epoch=272test accuracy=0.03125train accuracy=0.03125,learning rate=8.7261093e-10
epoch=273test accuracy=0.03125train accuracy=0.03125,learning rate=8.289804e-10
epoch=274test accuracy=0.03125train accuracy=0.03125,learning rate=7.875314e-10
epoch=275test accuracy=0.03125train accuracy=0.03125,learning rate=7.4815476e-10
epoch=276test accuracy=0.03125train accuracy=0.03125,learning rate=7.10747e-10
epoch=277test accuracy=0.03125train accuracy=0.03125,learning rate=6.7520967e-10
epoch=278test accuracy=0.03125train accuracy=0.03125,learning rate=6.414492e-10
epoch=279test accuracy=0.03125train accuracy=0.03125,learning rate=6.093767e-10
epoch=280test accuracy=0.03125train accuracy=0.03125,learning rate=5.789079e-10
epoch=281test accuracy=0.03125train accuracy=0.03125,learning rate=5.499625e-10
epoch=282test accuracy=0.03125train accuracy=0.03125,learning rate=5.224644e-10
epoch=283test accuracy=0.03125train accuracy=0.03125,learning rate=4.963412e-10
epoch=284test accuracy=0.03125train accuracy=0.03125,learning rate=4.715241e-10
epoch=285test accuracy=0.03125train accuracy=0.03125,learning rate=4.479479e-10
epoch=286test accuracy=0.03125train accuracy=0.03125,learning rate=4.255505e-10
epoch=287test accuracy=0.03125train accuracy=0.03125,learning rate=4.0427298e-10
epoch=288test accuracy=0.03125train accuracy=0.03125,learning rate=3.8405934e-10
epoch=289test accuracy=0.03125train accuracy=0.03125,learning rate=3.6485637e-10
epoch=290test accuracy=0.03125train accuracy=0.03125,learning rate=3.4661354e-10
epoch=291test accuracy=0.03125train accuracy=0.03125,learning rate=3.2928288e-10
epoch=292test accuracy=0.03125train accuracy=0.03125,learning rate=3.1281872e-10
epoch=293test accuracy=0.03125train accuracy=0.03125,learning rate=2.9717778e-10
epoch=294test accuracy=0.03125train accuracy=0.03125,learning rate=2.823189e-10
epoch=295test accuracy=0.03125train accuracy=0.03125,learning rate=2.6820296e-10
epoch=296test accuracy=0.03125train accuracy=0.03125,learning rate=2.547928e-10
epoch=297test accuracy=0.03125train accuracy=0.03125,learning rate=2.4205316e-10
epoch=298test accuracy=0.03125train accuracy=0.03125,learning rate=2.299505e-10
epoch=299test accuracy=0.03125train accuracy=0.03125,learning rate=2.1845298e-10
epoch=300test accuracy=0.03125train accuracy=0.03125,learning rate=2.0753034e-10
epoch=301test accuracy=0.03125train accuracy=0.03125,learning rate=1.9715382e-10
epoch=302test accuracy=0.03125train accuracy=0.03125,learning rate=1.8729612e-10
epoch=303test accuracy=0.03125train accuracy=0.03125,learning rate=1.7793132e-10
epoch=304test accuracy=0.03125train accuracy=0.03125,learning rate=1.6903476e-10
epoch=305test accuracy=0.03125train accuracy=0.03125,learning rate=1.6058302e-10
epoch=306test accuracy=0.03125train accuracy=0.03125,learning rate=1.5255387e-10
epoch=307test accuracy=0.03125train accuracy=0.03125,learning rate=1.4492617e-10
epoch=308test accuracy=0.03125train accuracy=0.03125,learning rate=1.3767987e-10
epoch=309test accuracy=0.03125train accuracy=0.03125,learning rate=1.3079587e-10
epoch=310test accuracy=0.03125train accuracy=0.03125,learning rate=1.2425608e-10
epoch=311test accuracy=0.03125train accuracy=0.03125,learning rate=1.1804327e-10
epoch=312test accuracy=0.03125train accuracy=0.03125,learning rate=1.1214111e-10
epoch=313test accuracy=0.03125train accuracy=0.03125,learning rate=1.0653405e-10
epoch=314test accuracy=0.03125train accuracy=0.03125,learning rate=1.01207355e-10
epoch=315test accuracy=0.03125train accuracy=0.03125,learning rate=9.6146986e-11
epoch=316test accuracy=0.03125train accuracy=0.03125,learning rate=9.133964e-11
epoch=317test accuracy=0.03125train accuracy=0.03125,learning rate=8.677265e-11
epoch=318test accuracy=0.03125train accuracy=0.03125,learning rate=8.243402e-11
epoch=319test accuracy=0.03125train accuracy=0.03125,learning rate=7.831232e-11
epoch=320test accuracy=0.03125train accuracy=0.03125,learning rate=7.4396704e-11
epoch=321test accuracy=0.03125train accuracy=0.03125,learning rate=7.067687e-11
epoch=322test accuracy=0.03125train accuracy=0.03125,learning rate=6.714303e-11
epoch=323test accuracy=0.03125train accuracy=0.03125,learning rate=6.378587e-11
epoch=324test accuracy=0.03125train accuracy=0.03125,learning rate=6.059658e-11
epoch=325test accuracy=0.03125train accuracy=0.03125,learning rate=5.756675e-11
epoch=326test accuracy=0.03125train accuracy=0.03125,learning rate=5.4688414e-11
epoch=327test accuracy=0.03125train accuracy=0.03125,learning rate=5.1953993e-11
epoch=328test accuracy=0.03125train accuracy=0.03125,learning rate=4.9356293e-11
epoch=329test accuracy=0.03125train accuracy=0.03125,learning rate=4.688848e-11
epoch=330test accuracy=0.03125train accuracy=0.03125,learning rate=4.4544056e-11
epoch=331test accuracy=0.03125train accuracy=0.03125,learning rate=4.231685e-11
epoch=332test accuracy=0.03125train accuracy=0.03125,learning rate=4.020101e-11
epoch=333test accuracy=0.03125train accuracy=0.03125,learning rate=3.8190957e-11
epoch=334test accuracy=0.03125train accuracy=0.03125,learning rate=3.6281412e-11
epoch=335test accuracy=0.03125train accuracy=0.03125,learning rate=3.446734e-11
epoch=336test accuracy=0.03125train accuracy=0.03125,learning rate=3.2743974e-11
epoch=337test accuracy=0.03125train accuracy=0.03125,learning rate=3.1106773e-11
epoch=338test accuracy=0.03125train accuracy=0.03125,learning rate=2.9551434e-11
epoch=339test accuracy=0.03125train accuracy=0.03125,learning rate=2.8073864e-11
epoch=340test accuracy=0.03125train accuracy=0.03125,learning rate=2.6670171e-11
epoch=341test accuracy=0.03125train accuracy=0.03125,learning rate=2.5336662e-11
epoch=342test accuracy=0.03125train accuracy=0.03125,learning rate=2.406983e-11
epoch=343test accuracy=0.03125train accuracy=0.03125,learning rate=2.2866337e-11
epoch=344test accuracy=0.03125train accuracy=0.03125,learning rate=2.1723021e-11
epoch=345test accuracy=0.03125train accuracy=0.03125,learning rate=2.063687e-11
epoch=346test accuracy=0.03125train accuracy=0.03125,learning rate=1.9605026e-11
epoch=347test accuracy=0.03125train accuracy=0.03125,learning rate=1.8624775e-11
epoch=348test accuracy=0.03125train accuracy=0.03125,learning rate=1.7693536e-11
epoch=349test accuracy=0.03125train accuracy=0.03125,learning rate=1.680886e-11
epoch=350test accuracy=0.03125train accuracy=0.03125,learning rate=1.5968416e-11
epoch=351test accuracy=0.03125train accuracy=0.03125,learning rate=1.5169995e-11
epoch=352test accuracy=0.03125train accuracy=0.03125,learning rate=1.4411496e-11
epoch=353test accuracy=0.03125train accuracy=0.03125,learning rate=1.3690921e-11
epoch=354test accuracy=0.03125train accuracy=0.03125,learning rate=1.3006375e-11
epoch=355test accuracy=0.03125train accuracy=0.03125,learning rate=1.2356056e-11
epoch=356test accuracy=0.03125train accuracy=0.03125,learning rate=1.1738254e-11
epoch=357test accuracy=0.03125train accuracy=0.03125,learning rate=1.1151341e-11
epoch=358test accuracy=0.03125train accuracy=0.03125,learning rate=1.0593774e-11
epoch=359test accuracy=0.03125train accuracy=0.03125,learning rate=1.0064085e-11
epoch=360test accuracy=0.03125train accuracy=0.03125,learning rate=9.560881e-12
epoch=361test accuracy=0.03125train accuracy=0.03125,learning rate=9.082837e-12
epoch=362test accuracy=0.03125train accuracy=0.03125,learning rate=8.628695e-12
epoch=363test accuracy=0.03125train accuracy=0.03125,learning rate=8.197261e-12
epoch=364test accuracy=0.03125train accuracy=0.03125,learning rate=7.7873975e-12
epoch=365test accuracy=0.03125train accuracy=0.03125,learning rate=7.3980275e-12
epoch=366test accuracy=0.03125train accuracy=0.03125,learning rate=7.028126e-12
epoch=367test accuracy=0.03125train accuracy=0.03125,learning rate=6.6767195e-12
epoch=368test accuracy=0.03125train accuracy=0.03125,learning rate=6.3428837e-12
epoch=369test accuracy=0.03125train accuracy=0.03125,learning rate=6.0257394e-12
epoch=370test accuracy=0.03125train accuracy=0.03125,learning rate=5.7244526e-12
epoch=371test accuracy=0.03125train accuracy=0.03125,learning rate=5.4382297e-12
epoch=372test accuracy=0.03125train accuracy=0.03125,learning rate=5.1663183e-12
epoch=373test accuracy=0.03125train accuracy=0.03125,learning rate=4.9080024e-12
epoch=374test accuracy=0.03125train accuracy=0.03125,learning rate=4.6626023e-12
epoch=375test accuracy=0.03125train accuracy=0.03125,learning rate=4.4294724e-12
epoch=376test accuracy=0.03125train accuracy=0.03125,learning rate=4.2079985e-12
epoch=377test accuracy=0.03125train accuracy=0.03125,learning rate=3.9975987e-12
epoch=378test accuracy=0.03125train accuracy=0.03125,learning rate=3.7977186e-12
epoch=379test accuracy=0.03125train accuracy=0.03125,learning rate=3.6078328e-12
epoch=380test accuracy=0.03125train accuracy=0.03125,learning rate=3.4274413e-12
epoch=381test accuracy=0.03125train accuracy=0.03125,learning rate=3.256069e-12
epoch=382test accuracy=0.03125train accuracy=0.03125,learning rate=3.0932657e-12
epoch=383test accuracy=0.03125train accuracy=0.03125,learning rate=2.9386023e-12
epoch=384test accuracy=0.03125train accuracy=0.03125,learning rate=2.7916723e-12
epoch=385test accuracy=0.03125train accuracy=0.03125,learning rate=2.6520886e-12
epoch=386test accuracy=0.03125train accuracy=0.03125,learning rate=2.5194842e-12
epoch=387test accuracy=0.03125train accuracy=0.03125,learning rate=2.39351e-12
epoch=388test accuracy=0.03125train accuracy=0.03125,learning rate=2.2738345e-12
epoch=389test accuracy=0.03125train accuracy=0.03125,learning rate=2.1601427e-12
epoch=390test accuracy=0.03125train accuracy=0.03125,learning rate=2.0521356e-12
epoch=391test accuracy=0.03125train accuracy=0.03125,learning rate=1.9495289e-12
epoch=392test accuracy=0.03125train accuracy=0.03125,learning rate=1.8520524e-12
epoch=393test accuracy=0.03125train accuracy=0.03125,learning rate=1.7594498e-12
epoch=394test accuracy=0.03125train accuracy=0.03125,learning rate=1.6714773e-12
epoch=395test accuracy=0.03125train accuracy=0.03125,learning rate=1.5879034e-12
epoch=396test accuracy=0.03125train accuracy=0.03125,learning rate=1.5085083e-12
epoch=397test accuracy=0.03125train accuracy=0.03125,learning rate=1.4330828e-12
epoch=398test accuracy=0.03125train accuracy=0.03125,learning rate=1.3614287e-12
epoch=399test accuracy=0.03125train accuracy=0.03125,learning rate=1.2933572e-12
epoch=400test accuracy=0.03125train accuracy=0.03125,learning rate=1.2286895e-12
epoch=401test accuracy=0.03125train accuracy=0.03125,learning rate=1.167255e-12
epoch=402test accuracy=0.03125train accuracy=0.03125,learning rate=1.1088922e-12
epoch=403test accuracy=0.03125train accuracy=0.03125,learning rate=1.0534476e-12
epoch=404test accuracy=0.03125train accuracy=0.03125,learning rate=1.0007752e-12
epoch=405test accuracy=0.03125train accuracy=0.03125,learning rate=9.507365e-13
epoch=406test accuracy=0.03125train accuracy=0.03125,learning rate=9.031996e-13
epoch=407test accuracy=0.03125train accuracy=0.03125,learning rate=8.5803966e-13
epoch=408test accuracy=0.03125train accuracy=0.03125,learning rate=8.1513767e-13
epoch=409test accuracy=0.03125train accuracy=0.03125,learning rate=7.743808e-13
epoch=410test accuracy=0.03125train accuracy=0.03125,learning rate=7.3566175e-13
epoch=411test accuracy=0.03125train accuracy=0.03125,learning rate=6.9887867e-13
epoch=412test accuracy=0.03125train accuracy=0.03125,learning rate=6.6393473e-13
epoch=413test accuracy=0.03125train accuracy=0.03125,learning rate=6.30738e-13
epoch=414test accuracy=0.03125train accuracy=0.03125,learning rate=5.992011e-13
epoch=415test accuracy=0.03125train accuracy=0.03125,learning rate=5.6924105e-13
epoch=416test accuracy=0.03125train accuracy=0.03125,learning rate=5.4077896e-13
epoch=417test accuracy=0.03125train accuracy=0.03125,learning rate=5.1374004e-13
epoch=418test accuracy=0.03125train accuracy=0.03125,learning rate=4.88053e-13
epoch=419test accuracy=0.03125train accuracy=0.03125,learning rate=4.636504e-13
epoch=420test accuracy=0.03125train accuracy=0.03125,learning rate=4.4046787e-13
epoch=421test accuracy=0.03125train accuracy=0.03125,learning rate=4.1844447e-13
epoch=422test accuracy=0.03125train accuracy=0.03125,learning rate=3.9752224e-13
epoch=423test accuracy=0.03125train accuracy=0.03125,learning rate=3.7764613e-13
epoch=424test accuracy=0.03125train accuracy=0.03125,learning rate=3.5876383e-13
epoch=425test accuracy=0.03125train accuracy=0.03125,learning rate=3.4082562e-13
epoch=426test accuracy=0.03125train accuracy=0.03125,learning rate=3.2378435e-13
epoch=427test accuracy=0.03125train accuracy=0.03125,learning rate=3.0759512e-13
epoch=428test accuracy=0.03125train accuracy=0.03125,learning rate=2.9221536e-13
epoch=429test accuracy=0.03125train accuracy=0.03125,learning rate=2.776046e-13
epoch=430test accuracy=0.03125train accuracy=0.03125,learning rate=2.6372437e-13
epoch=431test accuracy=0.03125train accuracy=0.03125,learning rate=2.5053817e-13
epoch=432test accuracy=0.03125train accuracy=0.03125,learning rate=2.3801124e-13
epoch=433test accuracy=0.03125train accuracy=0.03125,learning rate=2.2611069e-13
epoch=434test accuracy=0.03125train accuracy=0.03125,learning rate=2.1480516e-13
epoch=435test accuracy=0.03125train accuracy=0.03125,learning rate=2.040649e-13
epoch=436test accuracy=0.03125train accuracy=0.03125,learning rate=1.9386165e-13
epoch=437test accuracy=0.03125train accuracy=0.03125,learning rate=1.8416857e-13
epoch=438test accuracy=0.03125train accuracy=0.03125,learning rate=1.7496014e-13
epoch=439test accuracy=0.03125train accuracy=0.03125,learning rate=1.6621214e-13
epoch=440test accuracy=0.03125train accuracy=0.03125,learning rate=1.5790152e-13
epoch=441test accuracy=0.03125train accuracy=0.03125,learning rate=1.5000644e-13
epoch=442test accuracy=0.03125train accuracy=0.03125,learning rate=1.4250612e-13
epoch=443test accuracy=0.03125train accuracy=0.03125,learning rate=1.3538082e-13
epoch=444test accuracy=0.03125train accuracy=0.03125,learning rate=1.2861178e-13
epoch=445test accuracy=0.03125train accuracy=0.03125,learning rate=1.221812e-13
epoch=446test accuracy=0.03125train accuracy=0.03125,learning rate=1.1607214e-13
epoch=447test accuracy=0.03125train accuracy=0.03125,learning rate=1.10268524e-13
epoch=448test accuracy=0.03125train accuracy=0.03125,learning rate=1.047551e-13
epoch=449test accuracy=0.03125train accuracy=0.03125,learning rate=9.9517345e-14
epoch=450test accuracy=0.03125train accuracy=0.03125,learning rate=9.4541474e-14
epoch=451test accuracy=0.03125train accuracy=0.03125,learning rate=8.98144e-14
epoch=452test accuracy=0.03125train accuracy=0.03125,learning rate=8.532368e-14
epoch=453test accuracy=0.03125train accuracy=0.03125,learning rate=8.1057496e-14
epoch=454test accuracy=0.03125train accuracy=0.03125,learning rate=7.7004626e-14
epoch=455test accuracy=0.03125train accuracy=0.03125,learning rate=7.3154394e-14
epoch=456test accuracy=0.03125train accuracy=0.03125,learning rate=6.9496675e-14
epoch=457test accuracy=0.03125train accuracy=0.03125,learning rate=6.602184e-14
epoch=458test accuracy=0.03125train accuracy=0.03125,learning rate=6.272075e-14
epoch=459test accuracy=0.03125train accuracy=0.03125,learning rate=5.958471e-14
epoch=460test accuracy=0.03125train accuracy=0.03125,learning rate=5.6605474e-14
epoch=461test accuracy=0.03125train accuracy=0.03125,learning rate=5.37752e-14
epoch=462test accuracy=0.03125train accuracy=0.03125,learning rate=5.108644e-14
epoch=463test accuracy=0.03125train accuracy=0.03125,learning rate=4.8532118e-14
epoch=464test accuracy=0.03125train accuracy=0.03125,learning rate=4.610551e-14
epoch=465test accuracy=0.03125train accuracy=0.03125,learning rate=4.3800236e-14
epoch=466test accuracy=0.03125train accuracy=0.03125,learning rate=4.1610226e-14
epoch=467test accuracy=0.03125train accuracy=0.03125,learning rate=3.9529713e-14
epoch=468test accuracy=0.03125train accuracy=0.03125,learning rate=3.755323e-14
epoch=469test accuracy=0.03125train accuracy=0.03125,learning rate=3.5675567e-14
epoch=470test accuracy=0.03125train accuracy=0.03125,learning rate=3.3891787e-14
epoch=471test accuracy=0.03125train accuracy=0.03125,learning rate=3.21972e-14
epoch=472test accuracy=0.03125train accuracy=0.03125,learning rate=3.058734e-14
epoch=473test accuracy=0.03125train accuracy=0.03125,learning rate=2.9057973e-14
epoch=474test accuracy=0.03125train accuracy=0.03125,learning rate=2.7605073e-14
epoch=475test accuracy=0.03125train accuracy=0.03125,learning rate=2.622482e-14
epoch=476test accuracy=0.03125train accuracy=0.03125,learning rate=2.4913578e-14
epoch=477test accuracy=0.03125train accuracy=0.03125,learning rate=2.36679e-14
epoch=478test accuracy=0.03125train accuracy=0.03125,learning rate=2.2484505e-14
epoch=479test accuracy=0.03125train accuracy=0.03125,learning rate=2.136028e-14
epoch=480test accuracy=0.03125train accuracy=0.03125,learning rate=2.0292265e-14
epoch=481test accuracy=0.03125train accuracy=0.03125,learning rate=1.9277652e-14
epoch=482test accuracy=0.03125train accuracy=0.03125,learning rate=1.831377e-14
epoch=483test accuracy=0.03125train accuracy=0.03125,learning rate=1.739808e-14
epoch=484test accuracy=0.03125train accuracy=0.03125,learning rate=1.6528178e-14
epoch=485test accuracy=0.03125train accuracy=0.03125,learning rate=1.5701768e-14
epoch=486test accuracy=0.03125train accuracy=0.03125,learning rate=1.491668e-14
epoch=487test accuracy=0.03125train accuracy=0.03125,learning rate=1.4170846e-14
epoch=488test accuracy=0.03125train accuracy=0.03125,learning rate=1.3462304e-14
epoch=489test accuracy=0.03125train accuracy=0.03125,learning rate=1.2789189e-14
epoch=490test accuracy=0.03125train accuracy=0.03125,learning rate=1.2149729e-14
epoch=491test accuracy=0.03125train accuracy=0.03125,learning rate=1.1542243e-14
epoch=492test accuracy=0.03125train accuracy=0.03125,learning rate=1.096513e-14
epoch=493test accuracy=0.03125train accuracy=0.03125,learning rate=1.0416874e-14
epoch=494test accuracy=0.03125train accuracy=0.03125,learning rate=9.89603e-15
epoch=495test accuracy=0.03125train accuracy=0.03125,learning rate=9.401229e-15
epoch=496test accuracy=0.03125train accuracy=0.03125,learning rate=8.931167e-15
epoch=497test accuracy=0.03125train accuracy=0.03125,learning rate=8.484609e-15
epoch=498test accuracy=0.03125train accuracy=0.03125,learning rate=8.060378e-15
epoch=499test accuracy=0.03125train accuracy=0.03125,learning rate=7.65736e-15
epoch=500test accuracy=0.03125train accuracy=0.03125,learning rate=7.274492e-15
epoch=501test accuracy=0.03125train accuracy=0.03125,learning rate=6.910767e-15
epoch=502test accuracy=0.03125train accuracy=0.03125,learning rate=6.5652287e-15
epoch=503test accuracy=0.03125train accuracy=0.03125,learning rate=6.236967e-15
epoch=504test accuracy=0.03125train accuracy=0.03125,learning rate=5.9251187e-15
epoch=505test accuracy=0.03125train accuracy=0.03125,learning rate=5.628863e-15
epoch=506test accuracy=0.03125train accuracy=0.03125,learning rate=5.34742e-15
epoch=507test accuracy=0.03125train accuracy=0.03125,learning rate=5.0800488e-15
epoch=508test accuracy=0.03125train accuracy=0.03125,learning rate=4.8260464e-15
epoch=509test accuracy=0.03125train accuracy=0.03125,learning rate=4.584744e-15
epoch=510test accuracy=0.03125train accuracy=0.03125,learning rate=4.355507e-15
epoch=511test accuracy=0.03125train accuracy=0.03125,learning rate=4.1377314e-15
epoch=512test accuracy=0.03125train accuracy=0.03125,learning rate=3.930845e-15
epoch=513test accuracy=0.03125train accuracy=0.03125,learning rate=3.7343027e-15
epoch=514test accuracy=0.03125train accuracy=0.03125,learning rate=3.5475874e-15
epoch=515test accuracy=0.03125train accuracy=0.03125,learning rate=3.3702081e-15
epoch=516test accuracy=0.03125train accuracy=0.03125,learning rate=3.2016977e-15
epoch=517test accuracy=0.03125train accuracy=0.03125,learning rate=3.041613e-15
epoch=518test accuracy=0.03125train accuracy=0.03125,learning rate=2.8895322e-15
epoch=519test accuracy=0.03125train accuracy=0.03125,learning rate=2.7450557e-15
epoch=520test accuracy=0.03125train accuracy=0.03125,learning rate=2.607803e-15
epoch=521test accuracy=0.03125train accuracy=0.03125,learning rate=2.4774128e-15
epoch=522test accuracy=0.03125train accuracy=0.03125,learning rate=2.353542e-15
epoch=523test accuracy=0.03125train accuracy=0.03125,learning rate=2.235865e-15
epoch=524test accuracy=0.03125train accuracy=0.03125,learning rate=2.1240717e-15
epoch=525test accuracy=0.03125train accuracy=0.03125,learning rate=2.0178682e-15
epoch=526test accuracy=0.03125train accuracy=0.03125,learning rate=1.9169747e-15
epoch=527test accuracy=0.03125train accuracy=0.03125,learning rate=1.821126e-15
epoch=528test accuracy=0.03125train accuracy=0.03125,learning rate=1.7300697e-15
epoch=529test accuracy=0.03125train accuracy=0.03125,learning rate=1.6435662e-15
epoch=530test accuracy=0.03125train accuracy=0.03125,learning rate=1.5613879e-15
epoch=531test accuracy=0.03125train accuracy=0.03125,learning rate=1.4833185e-15
epoch=532test accuracy=0.03125train accuracy=0.03125,learning rate=1.4091526e-15
epoch=533test accuracy=0.03125train accuracy=0.03125,learning rate=1.3386949e-15
epoch=534test accuracy=0.03125train accuracy=0.03125,learning rate=1.2717601e-15
epoch=535test accuracy=0.03125train accuracy=0.03125,learning rate=1.2081722e-15
epoch=536test accuracy=0.03125train accuracy=0.03125,learning rate=1.1477635e-15
epoch=537test accuracy=0.03125train accuracy=0.03125,learning rate=1.0903753e-15
epoch=538test accuracy=0.03125train accuracy=0.03125,learning rate=1.0358566e-15
epoch=539test accuracy=0.03125train accuracy=0.03125,learning rate=9.840638e-16
epoch=540test accuracy=0.03125train accuracy=0.03125,learning rate=9.348606e-16
epoch=541test accuracy=0.03125train accuracy=0.03125,learning rate=8.881176e-16
epoch=542test accuracy=0.03125train accuracy=0.03125,learning rate=8.437117e-16
epoch=543test accuracy=0.03125train accuracy=0.03125,learning rate=8.015261e-16
epoch=544test accuracy=0.03125train accuracy=0.03125,learning rate=7.614498e-16
epoch=545test accuracy=0.03125train accuracy=0.03125,learning rate=7.233773e-16
epoch=546test accuracy=0.03125train accuracy=0.03125,learning rate=6.8720847e-16
epoch=547test accuracy=0.03125train accuracy=0.03125,learning rate=6.52848e-16
epoch=548test accuracy=0.03125train accuracy=0.03125,learning rate=6.202056e-16
epoch=549test accuracy=0.03125train accuracy=0.03125,learning rate=5.891953e-16
epoch=550test accuracy=0.03125train accuracy=0.03125,learning rate=5.5973557e-16
epoch=551test accuracy=0.03125train accuracy=0.03125,learning rate=5.317488e-16
epoch=552test accuracy=0.03125train accuracy=0.03125,learning rate=5.0516134e-16
epoch=553test accuracy=0.03125train accuracy=0.03125,learning rate=4.799033e-16
epoch=554test accuracy=0.03125train accuracy=0.03125,learning rate=4.559081e-16
epoch=555test accuracy=0.03125train accuracy=0.03125,learning rate=4.3311272e-16
epoch=556test accuracy=0.03125train accuracy=0.03125,learning rate=4.1145708e-16
epoch=557test accuracy=0.03125train accuracy=0.03125,learning rate=3.9088424e-16
epoch=558test accuracy=0.03125train accuracy=0.03125,learning rate=3.7134e-16
epoch=559test accuracy=0.03125train accuracy=0.03125,learning rate=3.5277302e-16
epoch=560test accuracy=0.03125train accuracy=0.03125,learning rate=3.3513436e-16
epoch=561test accuracy=0.03125train accuracy=0.03125,learning rate=3.1837764e-16
epoch=562test accuracy=0.03125train accuracy=0.03125,learning rate=3.0245876e-16
epoch=563test accuracy=0.03125train accuracy=0.03125,learning rate=2.8733584e-16
epoch=564test accuracy=0.03125train accuracy=0.03125,learning rate=2.7296905e-16
epoch=565test accuracy=0.03125train accuracy=0.03125,learning rate=2.593206e-16
epoch=566test accuracy=0.03125train accuracy=0.03125,learning rate=2.4635456e-16
epoch=567test accuracy=0.03125train accuracy=0.03125,learning rate=2.3403682e-16
epoch=568test accuracy=0.03125train accuracy=0.03125,learning rate=2.2233498e-16
epoch=569test accuracy=0.03125train accuracy=0.03125,learning rate=2.1121824e-16
epoch=570test accuracy=0.03125train accuracy=0.03125,learning rate=2.0065733e-16
epoch=571test accuracy=0.03125train accuracy=0.03125,learning rate=1.9062446e-16
epoch=572test accuracy=0.03125train accuracy=0.03125,learning rate=1.8109324e-16
epoch=573test accuracy=0.03125train accuracy=0.03125,learning rate=1.7203857e-16
epoch=574test accuracy=0.03125train accuracy=0.03125,learning rate=1.6343665e-16
epoch=575test accuracy=0.03125train accuracy=0.03125,learning rate=1.552648e-16
epoch=576test accuracy=0.03125train accuracy=0.03125,learning rate=1.4750158e-16
epoch=577test accuracy=0.03125train accuracy=0.03125,learning rate=1.4012649e-16
epoch=578test accuracy=0.03125train accuracy=0.03125,learning rate=1.3312017e-16
epoch=579test accuracy=0.03125train accuracy=0.03125,learning rate=1.2646416e-16
epoch=580test accuracy=0.03125train accuracy=0.03125,learning rate=1.2014096e-16
epoch=581test accuracy=0.03125train accuracy=0.03125,learning rate=1.141339e-16
epoch=582test accuracy=0.03125train accuracy=0.03125,learning rate=1.0842721e-16
epoch=583test accuracy=0.03125train accuracy=0.03125,learning rate=1.0300585e-16
epoch=584test accuracy=0.03125train accuracy=0.03125,learning rate=9.785556e-17
epoch=585test accuracy=0.03125train accuracy=0.03125,learning rate=9.296278e-17
epoch=586test accuracy=0.03125train accuracy=0.03125,learning rate=8.831464e-17
epoch=587test accuracy=0.03125train accuracy=0.03125,learning rate=8.3898905e-17
epoch=588test accuracy=0.03125train accuracy=0.03125,learning rate=7.970396e-17
epoch=589test accuracy=0.03125train accuracy=0.03125,learning rate=7.5718763e-17
epoch=590test accuracy=0.03125train accuracy=0.03125,learning rate=7.1932824e-17
epoch=591test accuracy=0.03125train accuracy=0.03125,learning rate=6.833618e-17
epoch=592test accuracy=0.03125train accuracy=0.03125,learning rate=6.491938e-17
epoch=593test accuracy=0.03125train accuracy=0.03125,learning rate=6.167341e-17
epoch=594test accuracy=0.03125train accuracy=0.03125,learning rate=5.858974e-17
epoch=595test accuracy=0.03125train accuracy=0.03125,learning rate=5.566025e-17
epoch=596test accuracy=0.03125train accuracy=0.03125,learning rate=5.2877236e-17
epoch=597test accuracy=0.03125train accuracy=0.03125,learning rate=5.0233375e-17
epoch=598test accuracy=0.03125train accuracy=0.03125,learning rate=4.7721707e-17
epoch=599test accuracy=0.03125train accuracy=0.03125,learning rate=4.5335622e-17
epoch=600test accuracy=0.03125train accuracy=0.03125,learning rate=4.306884e-17
epoch=601test accuracy=0.03125train accuracy=0.03125,learning rate=4.0915397e-17
epoch=602test accuracy=0.03125train accuracy=0.03125,learning rate=3.8869627e-17
epoch=603test accuracy=0.03125train accuracy=0.03125,learning rate=3.6926147e-17
epoch=604test accuracy=0.03125train accuracy=0.03125,learning rate=3.507984e-17
epoch=605test accuracy=0.03125train accuracy=0.03125,learning rate=3.3325846e-17
epoch=606test accuracy=0.03125train accuracy=0.03125,learning rate=3.1659554e-17
epoch=607test accuracy=0.03125train accuracy=0.03125,learning rate=3.0076578e-17
epoch=608test accuracy=0.03125train accuracy=0.03125,learning rate=2.8572747e-17
epoch=609test accuracy=0.03125train accuracy=0.03125,learning rate=2.714411e-17
epoch=610test accuracy=0.03125train accuracy=0.03125,learning rate=2.5786906e-17
epoch=611test accuracy=0.03125train accuracy=0.03125,learning rate=2.449756e-17
epoch=612test accuracy=0.03125train accuracy=0.03125,learning rate=2.3272682e-17
epoch=613test accuracy=0.03125train accuracy=0.03125,learning rate=2.2109047e-17
epoch=614test accuracy=0.03125train accuracy=0.03125,learning rate=2.1003595e-17
epoch=615test accuracy=0.03125train accuracy=0.03125,learning rate=1.9953416e-17
epoch=616test accuracy=0.03125train accuracy=0.03125,learning rate=1.8955745e-17
epoch=617test accuracy=0.03125train accuracy=0.03125,learning rate=1.8007959e-17
epoch=618test accuracy=0.03125train accuracy=0.03125,learning rate=1.710756e-17
epoch=619test accuracy=0.03125train accuracy=0.03125,learning rate=1.6252182e-17
epoch=620test accuracy=0.03125train accuracy=0.03125,learning rate=1.5439573e-17
epoch=621test accuracy=0.03125train accuracy=0.03125,learning rate=1.4667595e-17
epoch=622test accuracy=0.03125train accuracy=0.03125,learning rate=1.3934215e-17
epoch=623test accuracy=0.03125train accuracy=0.03125,learning rate=1.3237504e-17
epoch=624test accuracy=0.03125train accuracy=0.03125,learning rate=1.25756285e-17
epoch=625test accuracy=0.03125train accuracy=0.03125,learning rate=1.1946847e-17
epoch=626test accuracy=0.03125train accuracy=0.03125,learning rate=1.13495045e-17
epoch=627test accuracy=0.03125train accuracy=0.03125,learning rate=1.078203e-17
epoch=628test accuracy=0.03125train accuracy=0.03125,learning rate=1.0242928e-17
epoch=629test accuracy=0.03125train accuracy=0.03125,learning rate=9.7307815e-18
epoch=630test accuracy=0.03125train accuracy=0.03125,learning rate=9.244243e-18
epoch=631test accuracy=0.03125train accuracy=0.03125,learning rate=8.78203e-18
epoch=632test accuracy=0.03125train accuracy=0.03125,learning rate=8.342929e-18
epoch=633test accuracy=0.03125train accuracy=0.03125,learning rate=7.925782e-18
epoch=634test accuracy=0.03125train accuracy=0.03125,learning rate=7.5294934e-18
epoch=635test accuracy=0.03125train accuracy=0.03125,learning rate=7.153019e-18
epoch=636test accuracy=0.03125train accuracy=0.03125,learning rate=6.7953677e-18
epoch=637test accuracy=0.03125train accuracy=0.03125,learning rate=6.455599e-18
epoch=638test accuracy=0.03125train accuracy=0.03125,learning rate=6.1328192e-18
epoch=639test accuracy=0.03125train accuracy=0.03125,learning rate=5.8261784e-18
epoch=640test accuracy=0.03125train accuracy=0.03125,learning rate=5.5348694e-18
epoch=641test accuracy=0.03125train accuracy=0.03125,learning rate=5.258126e-18
epoch=642test accuracy=0.03125train accuracy=0.03125,learning rate=4.9952196e-18
epoch=643test accuracy=0.03125train accuracy=0.03125,learning rate=4.7454587e-18
epoch=644test accuracy=0.03125train accuracy=0.03125,learning rate=4.5081856e-18
epoch=645test accuracy=0.03125train accuracy=0.03125,learning rate=4.2827764e-18
epoch=646test accuracy=0.03125train accuracy=0.03125,learning rate=4.0686376e-18
epoch=647test accuracy=0.03125train accuracy=0.03125,learning rate=3.8652057e-18
epoch=648test accuracy=0.03125train accuracy=0.03125,learning rate=3.6719453e-18
epoch=649test accuracy=0.03125train accuracy=0.03125,learning rate=3.488348e-18
epoch=650test accuracy=0.03125train accuracy=0.03125,learning rate=3.3139308e-18
epoch=651test accuracy=0.03125train accuracy=0.03125,learning rate=3.1482343e-18
epoch=652test accuracy=0.03125train accuracy=0.03125,learning rate=2.9908225e-18
epoch=653test accuracy=0.03125train accuracy=0.03125,learning rate=2.8412814e-18
epoch=654test accuracy=0.03125train accuracy=0.03125,learning rate=2.6992173e-18
epoch=655test accuracy=0.03125train accuracy=0.03125,learning rate=2.5642566e-18
epoch=656test accuracy=0.03125train accuracy=0.03125,learning rate=2.4360436e-18
epoch=657test accuracy=0.03125train accuracy=0.03125,learning rate=2.3142415e-18
epoch=658test accuracy=0.03125train accuracy=0.03125,learning rate=2.1985294e-18
epoch=659test accuracy=0.03125train accuracy=0.03125,learning rate=2.088603e-18
epoch=660test accuracy=0.03125train accuracy=0.03125,learning rate=1.9841728e-18
epoch=661test accuracy=0.03125train accuracy=0.03125,learning rate=1.884964e-18
epoch=662test accuracy=0.03125train accuracy=0.03125,learning rate=1.790716e-18
epoch=663test accuracy=0.03125train accuracy=0.03125,learning rate=1.7011802e-18
epoch=664test accuracy=0.03125train accuracy=0.03125,learning rate=1.6161212e-18
epoch=665test accuracy=0.03125train accuracy=0.03125,learning rate=1.535315e-18
epoch=666test accuracy=0.03125train accuracy=0.03125,learning rate=1.4585494e-18
epoch=667test accuracy=0.03125train accuracy=0.03125,learning rate=1.3856218e-18
epoch=668test accuracy=0.03125train accuracy=0.03125,learning rate=1.3163408e-18
epoch=669test accuracy=0.03125train accuracy=0.03125,learning rate=1.2505237e-18
epoch=670test accuracy=0.03125train accuracy=0.03125,learning rate=1.1879975e-18
epoch=671test accuracy=0.03125train accuracy=0.03125,learning rate=1.1285977e-18
epoch=672test accuracy=0.03125train accuracy=0.03125,learning rate=1.0721678e-18
epoch=673test accuracy=0.03125train accuracy=0.03125,learning rate=1.0185594e-18
epoch=674test accuracy=0.03125train accuracy=0.03125,learning rate=9.676314e-19
epoch=675test accuracy=0.03125train accuracy=0.03125,learning rate=9.192498e-19
epoch=676test accuracy=0.03125train accuracy=0.03125,learning rate=8.732873e-19
epoch=677test accuracy=0.03125train accuracy=0.03125,learning rate=8.29623e-19
epoch=678test accuracy=0.03125train accuracy=0.03125,learning rate=7.881418e-19
epoch=679test accuracy=0.03125train accuracy=0.03125,learning rate=7.4873473e-19
epoch=680test accuracy=0.03125train accuracy=0.03125,learning rate=7.1129804e-19
epoch=681test accuracy=0.03125train accuracy=0.03125,learning rate=6.757331e-19
epoch=682test accuracy=0.03125train accuracy=0.03125,learning rate=6.4194644e-19
epoch=683test accuracy=0.03125train accuracy=0.03125,learning rate=6.0984914e-19
epoch=684test accuracy=0.03125train accuracy=0.03125,learning rate=5.793567e-19
epoch=685test accuracy=0.03125train accuracy=0.03125,learning rate=5.5038887e-19
epoch=686test accuracy=0.03125train accuracy=0.03125,learning rate=5.228694e-19
epoch=687test accuracy=0.03125train accuracy=0.03125,learning rate=4.9672594e-19
epoch=688test accuracy=0.03125train accuracy=0.03125,learning rate=4.7188963e-19
epoch=689test accuracy=0.03125train accuracy=0.03125,learning rate=4.4829513e-19
epoch=690test accuracy=0.03125train accuracy=0.03125,learning rate=4.258804e-19
epoch=691test accuracy=0.03125train accuracy=0.03125,learning rate=4.0458637e-19
epoch=692test accuracy=0.03125train accuracy=0.03125,learning rate=3.8435705e-19
epoch=693test accuracy=0.03125train accuracy=0.03125,learning rate=3.651392e-19
epoch=694test accuracy=0.03125train accuracy=0.03125,learning rate=3.4688225e-19
epoch=695test accuracy=0.03125train accuracy=0.03125,learning rate=3.2953813e-19
epoch=696test accuracy=0.03125train accuracy=0.03125,learning rate=3.1306124e-19
epoch=697test accuracy=0.03125train accuracy=0.03125,learning rate=2.9740816e-19
epoch=698test accuracy=0.03125train accuracy=0.03125,learning rate=2.8253776e-19
epoch=699test accuracy=0.03125train accuracy=0.03125,learning rate=2.6841088e-19
epoch=700test accuracy=0.03125train accuracy=0.03125,learning rate=2.5499031e-19
epoch=701test accuracy=0.03125train accuracy=0.03125,learning rate=2.422408e-19
epoch=702test accuracy=0.03125train accuracy=0.03125,learning rate=2.3012878e-19
epoch=703test accuracy=0.03125train accuracy=0.03125,learning rate=2.1862234e-19
epoch=704test accuracy=0.03125train accuracy=0.03125,learning rate=2.0769121e-19
epoch=705test accuracy=0.03125train accuracy=0.03125,learning rate=1.9730665e-19
epoch=706test accuracy=0.03125train accuracy=0.03125,learning rate=1.8744132e-19
epoch=707test accuracy=0.03125train accuracy=0.03125,learning rate=1.7806925e-19
epoch=708test accuracy=0.03125train accuracy=0.03125,learning rate=1.6916579e-19
epoch=709test accuracy=0.03125train accuracy=0.03125,learning rate=1.607075e-19
epoch=710test accuracy=0.03125train accuracy=0.03125,learning rate=1.5267213e-19
epoch=711test accuracy=0.03125train accuracy=0.03125,learning rate=1.4503852e-19
epoch=712test accuracy=0.03125train accuracy=0.03125,learning rate=1.3778659e-19
epoch=713test accuracy=0.03125train accuracy=0.03125,learning rate=1.3089726e-19
epoch=714test accuracy=0.03125train accuracy=0.03125,learning rate=1.243524e-19
epoch=715test accuracy=0.03125train accuracy=0.03125,learning rate=1.1813478e-19
epoch=716test accuracy=0.03125train accuracy=0.03125,learning rate=1.1222804e-19
epoch=717test accuracy=0.03125train accuracy=0.03125,learning rate=1.0661664e-19
epoch=718test accuracy=0.03125train accuracy=0.03125,learning rate=1.0128581e-19
epoch=719test accuracy=0.03125train accuracy=0.03125,learning rate=9.622152e-20
epoch=720test accuracy=0.03125train accuracy=0.03125,learning rate=9.1410443e-20
epoch=721test accuracy=0.03125train accuracy=0.03125,learning rate=8.683992e-20
epoch=722test accuracy=0.03125train accuracy=0.03125,learning rate=8.249792e-20
epoch=723test accuracy=0.03125train accuracy=0.03125,learning rate=7.837303e-20
epoch=724test accuracy=0.03125train accuracy=0.03125,learning rate=7.445438e-20
epoch=725test accuracy=0.03125train accuracy=0.03125,learning rate=7.073166e-20
epoch=726test accuracy=0.03125train accuracy=0.03125,learning rate=6.7195074e-20
epoch=727test accuracy=0.03125train accuracy=0.03125,learning rate=6.383532e-20
epoch=728test accuracy=0.03125train accuracy=0.03125,learning rate=6.064356e-20
epoch=729test accuracy=0.03125train accuracy=0.03125,learning rate=5.761138e-20
epoch=730test accuracy=0.03125train accuracy=0.03125,learning rate=5.4730806e-20
epoch=731test accuracy=0.03125train accuracy=0.03125,learning rate=5.1994266e-20
epoch=732test accuracy=0.03125train accuracy=0.03125,learning rate=4.9394554e-20
epoch=733test accuracy=0.03125train accuracy=0.03125,learning rate=4.6924825e-20
epoch=734test accuracy=0.03125train accuracy=0.03125,learning rate=4.4578586e-20
epoch=735test accuracy=0.03125train accuracy=0.03125,learning rate=4.2349657e-20
epoch=736test accuracy=0.03125train accuracy=0.03125,learning rate=4.0232175e-20
epoch=737test accuracy=0.03125train accuracy=0.03125,learning rate=3.8220565e-20
epoch=738test accuracy=0.03125train accuracy=0.03125,learning rate=3.6309536e-20
epoch=739test accuracy=0.03125train accuracy=0.03125,learning rate=3.449406e-20
epoch=740test accuracy=0.03125train accuracy=0.03125,learning rate=3.2769358e-20
epoch=741test accuracy=0.03125train accuracy=0.03125,learning rate=3.1130887e-20
epoch=742test accuracy=0.03125train accuracy=0.03125,learning rate=2.9574344e-20
epoch=743test accuracy=0.03125train accuracy=0.03125,learning rate=2.8095626e-20
epoch=744test accuracy=0.03125train accuracy=0.03125,learning rate=2.6690845e-20
epoch=745test accuracy=0.03125train accuracy=0.03125,learning rate=2.5356304e-20
epoch=746test accuracy=0.03125train accuracy=0.03125,learning rate=2.4088488e-20
epoch=747test accuracy=0.03125train accuracy=0.03125,learning rate=2.2884063e-20
epoch=748test accuracy=0.03125train accuracy=0.03125,learning rate=2.173986e-20
epoch=749test accuracy=0.03125train accuracy=0.03125,learning rate=2.0652868e-20
epoch=750test accuracy=0.03125train accuracy=0.03125,learning rate=1.9620225e-20
epoch=751test accuracy=0.03125train accuracy=0.03125,learning rate=1.8639213e-20
epoch=752test accuracy=0.03125train accuracy=0.03125,learning rate=1.7707252e-20
epoch=753test accuracy=0.03125train accuracy=0.03125,learning rate=1.6821889e-20
epoch=754test accuracy=0.03125train accuracy=0.03125,learning rate=1.5980795e-20
epoch=755test accuracy=0.03125train accuracy=0.03125,learning rate=1.5181756e-20
epoch=756test accuracy=0.03125train accuracy=0.03125,learning rate=1.4422668e-20
epoch=757test accuracy=0.03125train accuracy=0.03125,learning rate=1.3701534e-20
epoch=758test accuracy=0.03125train accuracy=0.03125,learning rate=1.30164576e-20
epoch=759test accuracy=0.03125train accuracy=0.03125,learning rate=1.23656345e-20
epoch=760test accuracy=0.03125train accuracy=0.03125,learning rate=1.1747353e-20
epoch=761test accuracy=0.03125train accuracy=0.03125,learning rate=1.11599856e-20
epoch=762test accuracy=0.03125train accuracy=0.03125,learning rate=1.0601986e-20
epoch=763test accuracy=0.03125train accuracy=0.03125,learning rate=1.0071887e-20
epoch=764test accuracy=0.03125train accuracy=0.03125,learning rate=9.568292e-21
epoch=765test accuracy=0.03125train accuracy=0.03125,learning rate=9.089878e-21
epoch=766test accuracy=0.03125train accuracy=0.03125,learning rate=8.635384e-21
epoch=767test accuracy=0.03125train accuracy=0.03125,learning rate=8.203615e-21
epoch=768test accuracy=0.03125train accuracy=0.03125,learning rate=7.793434e-21
epoch=769test accuracy=0.03125train accuracy=0.03125,learning rate=7.4037625e-21
epoch=770test accuracy=0.03125train accuracy=0.03125,learning rate=7.033574e-21
epoch=771test accuracy=0.03125train accuracy=0.03125,learning rate=6.6818955e-21
epoch=772test accuracy=0.03125train accuracy=0.03125,learning rate=6.3478006e-21
epoch=773test accuracy=0.03125train accuracy=0.03125,learning rate=6.030411e-21
epoch=774test accuracy=0.03125train accuracy=0.03125,learning rate=5.72889e-21
epoch=775test accuracy=0.03125train accuracy=0.03125,learning rate=5.4424457e-21
epoch=776test accuracy=0.03125train accuracy=0.03125,learning rate=5.1703235e-21
epoch=777test accuracy=0.03125train accuracy=0.03125,learning rate=4.9118074e-21
epoch=778test accuracy=0.03125train accuracy=0.03125,learning rate=4.6662167e-21
epoch=779test accuracy=0.03125train accuracy=0.03125,learning rate=4.432906e-21
epoch=780test accuracy=0.03125train accuracy=0.03125,learning rate=4.2112605e-21
epoch=781test accuracy=0.03125train accuracy=0.03125,learning rate=4.000698e-21
epoch=782test accuracy=0.03125train accuracy=0.03125,learning rate=3.8006627e-21
epoch=783test accuracy=0.03125train accuracy=0.03125,learning rate=3.6106296e-21
epoch=784test accuracy=0.03125train accuracy=0.03125,learning rate=3.4300983e-21
epoch=785test accuracy=0.03125train accuracy=0.03125,learning rate=3.2585932e-21
epoch=786test accuracy=0.03125train accuracy=0.03125,learning rate=3.0956637e-21
epoch=787test accuracy=0.03125train accuracy=0.03125,learning rate=2.9408803e-21
epoch=788test accuracy=0.03125train accuracy=0.03125,learning rate=2.7938364e-21
epoch=789test accuracy=0.03125train accuracy=0.03125,learning rate=2.6541445e-21
epoch=790test accuracy=0.03125train accuracy=0.03125,learning rate=2.5214373e-21
epoch=791test accuracy=0.03125train accuracy=0.03125,learning rate=2.3953654e-21
epoch=792test accuracy=0.03125train accuracy=0.03125,learning rate=2.275597e-21
epoch=793test accuracy=0.03125train accuracy=0.03125,learning rate=2.1618173e-21
epoch=794test accuracy=0.03125train accuracy=0.03125,learning rate=2.0537265e-21
epoch=795test accuracy=0.03125train accuracy=0.03125,learning rate=1.9510402e-21
epoch=796test accuracy=0.03125train accuracy=0.03125,learning rate=1.8534882e-21
epoch=797test accuracy=0.03125train accuracy=0.03125,learning rate=1.7608137e-21
epoch=798test accuracy=0.03125train accuracy=0.03125,learning rate=1.6727731e-21
epoch=799test accuracy=0.03125train accuracy=0.03125,learning rate=1.5891344e-21
epoch=800test accuracy=0.03125train accuracy=0.03125,learning rate=1.5096776e-21
epoch=801test accuracy=0.03125train accuracy=0.03125,learning rate=1.4341938e-21
epoch=802test accuracy=0.03125train accuracy=0.03125,learning rate=1.3624841e-21
epoch=803test accuracy=0.03125train accuracy=0.03125,learning rate=1.2943598e-21
epoch=804test accuracy=0.03125train accuracy=0.03125,learning rate=1.2296419e-21
epoch=805test accuracy=0.03125train accuracy=0.03125,learning rate=1.1681598e-21
epoch=806test accuracy=0.03125train accuracy=0.03125,learning rate=1.1097519e-21
epoch=807test accuracy=0.03125train accuracy=0.03125,learning rate=1.0542642e-21
epoch=808test accuracy=0.03125train accuracy=0.03125,learning rate=1.001551e-21
epoch=809test accuracy=0.03125train accuracy=0.03125,learning rate=9.514735e-22
epoch=810test accuracy=0.03125train accuracy=0.03125,learning rate=9.038998e-22
epoch=811test accuracy=0.03125train accuracy=0.03125,learning rate=8.587048e-22
epoch=812test accuracy=0.03125train accuracy=0.03125,learning rate=8.1576957e-22
epoch=813test accuracy=0.03125train accuracy=0.03125,learning rate=7.749811e-22
epoch=814test accuracy=0.03125train accuracy=0.03125,learning rate=7.3623204e-22
epoch=815test accuracy=0.03125train accuracy=0.03125,learning rate=6.9942044e-22
epoch=816test accuracy=0.03125train accuracy=0.03125,learning rate=6.644494e-22
epoch=817test accuracy=0.03125train accuracy=0.03125,learning rate=6.3122696e-22
epoch=818test accuracy=0.03125train accuracy=0.03125,learning rate=5.996656e-22
epoch=819test accuracy=0.03125train accuracy=0.03125,learning rate=5.6968233e-22
epoch=820test accuracy=0.03125train accuracy=0.03125,learning rate=5.411982e-22
epoch=821test accuracy=0.03125train accuracy=0.03125,learning rate=5.141383e-22
epoch=822test accuracy=0.03125train accuracy=0.03125,learning rate=4.8843137e-22
epoch=823test accuracy=0.03125train accuracy=0.03125,learning rate=4.640098e-22
epoch=824test accuracy=0.03125train accuracy=0.03125,learning rate=4.408093e-22
epoch=825test accuracy=0.03125train accuracy=0.03125,learning rate=4.1876884e-22
epoch=826test accuracy=0.03125train accuracy=0.03125,learning rate=3.978304e-22
epoch=827test accuracy=0.03125train accuracy=0.03125,learning rate=3.779389e-22

In [ ]:
#lr = 0.1*(0.95)**epoch
epoch=0,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.1
epoch=1,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.095
epoch=2,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.09025
epoch=3,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0857375
epoch=4,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.081450626
epoch=5,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.077378094
epoch=6,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.07350919
epoch=7,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.06983373
epoch=8,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.06634204
epoch=9,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.06302494
epoch=10,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.059873693
epoch=11,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.05688001
epoch=12,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.05403601
epoch=13,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.05133421
epoch=14,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0487675
epoch=15,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.046329122
epoch=16,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.044012666
epoch=17,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.041812032
epoch=18,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.039721433
epoch=19,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.03773536
epoch=20,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.03584859
epoch=21,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.034056164
epoch=22,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.032353353
epoch=23,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.030735686
epoch=24,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.029198902
epoch=25,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.027738957
epoch=26,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.026352009
epoch=27,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.025034409
epoch=28,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.02378269
epoch=29,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.022593554
epoch=30,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.021463877
epoch=31,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.020390682
epoch=32,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.019371148
epoch=33,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.018402591
epoch=34,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.017482461
epoch=35,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.016608339
epoch=36,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.015777921
epoch=37,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.014989025
epoch=38,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.014239574
epoch=39,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.013527595
epoch=40,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.012851216
epoch=41,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.012208655
epoch=42,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.011598222
epoch=43,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.011018311
epoch=44,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.010467395
epoch=45,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.009944025
epoch=46,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.009446824
epoch=47,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.008974483
epoch=48,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.008525759
epoch=49,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.008099471
epoch=50,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0076944977
epoch=51,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0073097725
epoch=52,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.006944284
epoch=53,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00659707
epoch=54,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0062672165
epoch=55,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0059538553
epoch=56,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0056561627
epoch=57,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0053733545
epoch=58,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.005104687
epoch=59,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0048494525
epoch=60,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00460698
epoch=61,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0043766308
epoch=62,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0041577993
epoch=63,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0039499095
epoch=64,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.003752414
epoch=65,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0035647932
epoch=66,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0033865536
epoch=67,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.003217226
epoch=68,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0030563646
epoch=69,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0029035464
epoch=70,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.002758369
epoch=71,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0026204507
epoch=72,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.002489428
epoch=73,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0023649568
epoch=74,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0022467088
epoch=75,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0021343734
epoch=76,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0020276548
epoch=77,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.001926272
epoch=78,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0018299584
epoch=79,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0017384605
epoch=80,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0016515375
epoch=81,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0015689606
epoch=82,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0014905125
epoch=83,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0014159869
epoch=84,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0013451875
epoch=85,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0012779282
epoch=86,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0012140318
epoch=87,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0011533302
epoch=88,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0010956636
epoch=89,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0010408805
epoch=90,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0009888365
epoch=91,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00093939464
epoch=92,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0008924249
epoch=93,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00084780366
epoch=94,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00080541347
epoch=95,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0007651428
epoch=96,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0007268857
epoch=97,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0006905414
epoch=98,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0006560143
epoch=99,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0006232136
epoch=100,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00059205294
epoch=101,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00056245027
epoch=102,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00053432776
epoch=103,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0005076114
epoch=104,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0004822308
epoch=105,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00045811926
epoch=106,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0004352133
epoch=107,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00041345262
epoch=108,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00039278
epoch=109,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.000373141
epoch=110,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00035448396
epoch=111,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00033675975
epoch=112,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00031992176
epoch=113,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00030392568
epoch=114,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0002887294
epoch=115,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00027429292
epoch=116,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0002605783
epoch=117,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00024754935
epoch=118,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0002351719
epoch=119,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0002234133
epoch=120,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00021224264
epoch=121,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00020163051
epoch=122,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00019154898
epoch=123,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00018197153
epoch=124,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00017287295
epoch=125,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0001642293
epoch=126,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00015601784
epoch=127,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00014821695
epoch=128,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0001408061
epoch=129,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.0001337658
epoch=130,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00012707751
epoch=131,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.000120723635
epoch=132,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00011468745
epoch=133,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00010895308
epoch=134,test accuracy=0.03125,train accuracy=0.03125,learning rate=0.00010350542
epoch=135,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.833015e-05
epoch=136,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.341365e-05
epoch=137,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.874296e-05
epoch=138,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.4305815e-05
epoch=139,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.0090525e-05
epoch=140,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.6086e-05
epoch=141,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.22817e-05
epoch=142,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.8667614e-05
epoch=143,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.5234235e-05
epoch=144,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.197252e-05
epoch=145,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.8873895e-05
epoch=146,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.59302e-05
epoch=147,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.313369e-05
epoch=148,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.0477007e-05
epoch=149,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.7953155e-05
epoch=150,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.55555e-05
epoch=151,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.3277723e-05
epoch=152,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.1113835e-05
epoch=153,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.9058144e-05
epoch=154,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.7105237e-05
epoch=155,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.5249974e-05
epoch=156,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.348748e-05
epoch=157,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.1813102e-05
epoch=158,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.0222447e-05
epoch=159,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.8711325e-05
epoch=160,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.7275759e-05
epoch=161,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5911972e-05
epoch=162,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4616373e-05
epoch=163,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.3385554e-05
epoch=164,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2216276e-05
epoch=165,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1105463e-05
epoch=166,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0050189e-05
epoch=167,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.904768e-05
epoch=168,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8095296e-05
epoch=169,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.719053e-05
epoch=170,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6331005e-05
epoch=171,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5514453e-05
epoch=172,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4738732e-05
epoch=173,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4001795e-05
epoch=174,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3301706e-05
epoch=175,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.263662e-05
epoch=176,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2004789e-05
epoch=177,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.140455e-05
epoch=178,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0834322e-05
epoch=179,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0292606e-05
epoch=180,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.777976e-06
epoch=181,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.289077e-06
epoch=182,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.824623e-06
epoch=183,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.383392e-06
epoch=184,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.964222e-06
epoch=185,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.5660114e-06
epoch=186,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.1877107e-06
epoch=187,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.8283252e-06
epoch=188,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.4869087e-06
epoch=189,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.1625633e-06
epoch=190,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.854435e-06
epoch=191,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.5617134e-06
epoch=192,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.2836276e-06
epoch=193,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.0194462e-06
epoch=194,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.768474e-06
epoch=195,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.5300503e-06
epoch=196,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.303548e-06
epoch=197,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.0883706e-06
epoch=198,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.883952e-06
epoch=199,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.6897543e-06
epoch=200,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.5052667e-06
epoch=201,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.3300032e-06
epoch=202,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.1635032e-06
epoch=203,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.005328e-06
epoch=204,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.8550617e-06
epoch=205,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.7123085e-06
epoch=206,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.576693e-06
epoch=207,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4478584e-06
epoch=208,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.3254654e-06
epoch=209,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2091922e-06
epoch=210,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0987327e-06
epoch=211,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9937959e-06
epoch=212,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8941062e-06
epoch=213,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7994008e-06
epoch=214,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7094309e-06
epoch=215,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6239593e-06
epoch=216,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5427613e-06
epoch=217,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4656232e-06
epoch=218,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3923421e-06
epoch=219,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.322725e-06
epoch=220,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2565887e-06
epoch=221,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1937593e-06
epoch=222,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1340713e-06
epoch=223,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0773678e-06
epoch=224,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0234994e-06
epoch=225,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.723244e-07
epoch=226,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.2370817e-07
epoch=227,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.775228e-07
epoch=228,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.3364665e-07
epoch=229,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.9196434e-07
epoch=230,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.523661e-07
epoch=231,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.147478e-07
epoch=232,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.790104e-07
epoch=233,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.4505986e-07
epoch=234,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.128069e-07
epoch=235,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.8216654e-07
epoch=236,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.530582e-07
epoch=237,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.254053e-07
epoch=238,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.99135e-07
epoch=239,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.7417828e-07
epoch=240,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.5046937e-07
epoch=241,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.279459e-07
epoch=242,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.0654862e-07
epoch=243,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.862212e-07
epoch=244,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.6691011e-07
epoch=245,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.485646e-07
epoch=246,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.311364e-07
epoch=247,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.1457955e-07
epoch=248,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.9885058e-07
epoch=249,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.8390807e-07
epoch=250,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6971264e-07
epoch=251,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5622703e-07
epoch=252,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4341568e-07
epoch=253,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.3124488e-07
epoch=254,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1968265e-07
epoch=255,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0869851e-07
epoch=256,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9826358e-07
epoch=257,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.883504e-07
epoch=258,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7893288e-07
epoch=259,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6998624e-07
epoch=260,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6148692e-07
epoch=261,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5341259e-07
epoch=262,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4574195e-07
epoch=263,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3845485e-07
epoch=264,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3153212e-07
epoch=265,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.249555e-07
epoch=266,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1870773e-07
epoch=267,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1277235e-07
epoch=268,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0713373e-07
epoch=269,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0177704e-07
epoch=270,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.668819e-08
epoch=271,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.185378e-08
epoch=272,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.726109e-08
epoch=273,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.289803e-08
epoch=274,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.875313e-08
epoch=275,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.4815475e-08
epoch=276,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.10747e-08
epoch=277,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.752097e-08
epoch=278,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.414492e-08
epoch=279,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.093767e-08
epoch=280,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.789079e-08
epoch=281,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.499625e-08
epoch=282,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.224644e-08
epoch=283,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.9634117e-08
epoch=284,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.715241e-08
epoch=285,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.4794792e-08
epoch=286,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.255505e-08
epoch=287,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.04273e-08
epoch=288,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.8405933e-08
epoch=289,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.6485638e-08
epoch=290,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.4661355e-08
epoch=291,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.2928288e-08
epoch=292,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.128187e-08
epoch=293,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.9717778e-08
epoch=294,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.823189e-08
epoch=295,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6820295e-08
epoch=296,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5479281e-08
epoch=297,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4205317e-08
epoch=298,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2995051e-08
epoch=299,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1845299e-08
epoch=300,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0753033e-08
epoch=301,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9715381e-08
epoch=302,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8729613e-08
epoch=303,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7793132e-08
epoch=304,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6903476e-08
epoch=305,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6058301e-08
epoch=306,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5255386e-08
epoch=307,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4492617e-08
epoch=308,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3767987e-08
epoch=309,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3079587e-08
epoch=310,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2425608e-08
epoch=311,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1804327e-08
epoch=312,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1214111e-08
epoch=313,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0653405e-08
epoch=314,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0120735e-08
epoch=315,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.614698e-09
epoch=316,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.133964e-09
epoch=317,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.677265e-09
epoch=318,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.243402e-09
epoch=319,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.831232e-09
epoch=320,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.43967e-09
epoch=321,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.067687e-09
epoch=322,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.7143024e-09
epoch=323,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.3785874e-09
epoch=324,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.059658e-09
epoch=325,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.756675e-09
epoch=326,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.4688414e-09
epoch=327,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.195399e-09
epoch=328,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.9356292e-09
epoch=329,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.688848e-09
epoch=330,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.4544053e-09
epoch=331,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.2316852e-09
epoch=332,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.020101e-09
epoch=333,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.8190957e-09
epoch=334,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.6281411e-09
epoch=335,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.446734e-09
epoch=336,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.2743974e-09
epoch=337,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.1106775e-09
epoch=338,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.9551437e-09
epoch=339,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.8073863e-09
epoch=340,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.667017e-09
epoch=341,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5336662e-09
epoch=342,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4069828e-09
epoch=343,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2866338e-09
epoch=344,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1723021e-09
epoch=345,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.063687e-09
epoch=346,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9605026e-09
epoch=347,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8624775e-09
epoch=348,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7693537e-09
epoch=349,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.680886e-09
epoch=350,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5968417e-09
epoch=351,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5169995e-09
epoch=352,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4411495e-09
epoch=353,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3690921e-09
epoch=354,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3006375e-09
epoch=355,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2356056e-09
epoch=356,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1738254e-09
epoch=357,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1151341e-09
epoch=358,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0593774e-09
epoch=359,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0064085e-09
epoch=360,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.560881e-10
epoch=361,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.0828367e-10
epoch=362,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.628695e-10
epoch=363,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.19726e-10
epoch=364,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.7873974e-10
epoch=365,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.398027e-10
epoch=366,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.028126e-10
epoch=367,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.67672e-10
epoch=368,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.3428834e-10
epoch=369,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.0257394e-10
epoch=370,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.7244526e-10
epoch=371,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.43823e-10
epoch=372,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.1663185e-10
epoch=373,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.9080023e-10
epoch=374,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.6626025e-10
epoch=375,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.4294723e-10
epoch=376,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.2079987e-10
epoch=377,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.9975986e-10
epoch=378,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.7977188e-10
epoch=379,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.607833e-10
epoch=380,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.427441e-10
epoch=381,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.256069e-10
epoch=382,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.0932656e-10
epoch=383,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.9386024e-10
epoch=384,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.7916722e-10
epoch=385,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6520885e-10
epoch=386,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5194843e-10
epoch=387,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.39351e-10
epoch=388,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2738345e-10
epoch=389,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1601428e-10
epoch=390,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0521357e-10
epoch=391,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9495289e-10
epoch=392,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8520524e-10
epoch=393,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7594498e-10
epoch=394,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6714773e-10
epoch=395,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5879034e-10
epoch=396,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5085083e-10
epoch=397,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4330828e-10
epoch=398,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3614287e-10
epoch=399,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2933572e-10
epoch=400,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2286894e-10
epoch=401,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1672549e-10
epoch=402,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1088922e-10
epoch=403,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0534476e-10
epoch=404,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0007752e-10
epoch=405,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.507364e-11
epoch=406,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.031996e-11
epoch=407,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.580396e-11
epoch=408,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.151377e-11
epoch=409,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.743808e-11
epoch=410,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.3566174e-11
epoch=411,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.9887866e-11
epoch=412,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.6393474e-11
epoch=413,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.3073796e-11
epoch=414,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.992011e-11
epoch=415,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.6924104e-11
epoch=416,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.40779e-11
epoch=417,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.1374002e-11
epoch=418,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.8805304e-11
epoch=419,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.6365037e-11
epoch=420,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.4046787e-11
epoch=421,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.1844448e-11
epoch=422,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.9752226e-11
epoch=423,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.7764614e-11
epoch=424,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.587638e-11
epoch=425,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.4082563e-11
epoch=426,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.2378433e-11
epoch=427,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.0759513e-11
epoch=428,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.922154e-11
epoch=429,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.776046e-11
epoch=430,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6372437e-11
epoch=431,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.5053815e-11
epoch=432,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.3801124e-11
epoch=433,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.261107e-11
epoch=434,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1480515e-11
epoch=435,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0406489e-11
epoch=436,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9386165e-11
epoch=437,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.8416857e-11
epoch=438,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7496013e-11
epoch=439,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6621213e-11
epoch=440,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5790152e-11
epoch=441,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5000645e-11
epoch=442,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4250613e-11
epoch=443,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3538082e-11
epoch=444,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2861178e-11
epoch=445,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2218119e-11
epoch=446,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.16072135e-11
epoch=447,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1026852e-11
epoch=448,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.047551e-11
epoch=449,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.951735e-12
epoch=450,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.4541475e-12
epoch=451,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.981441e-12
epoch=452,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.532368e-12
epoch=453,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.10575e-12
epoch=454,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.700463e-12
epoch=455,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.315439e-12
epoch=456,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.9496674e-12
epoch=457,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.602184e-12
epoch=458,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.272075e-12
epoch=459,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.958471e-12
epoch=460,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.6605476e-12
epoch=461,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.37752e-12
epoch=462,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.108644e-12
epoch=463,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.853212e-12
epoch=464,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.610551e-12
epoch=465,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.3800237e-12
epoch=466,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.1610227e-12
epoch=467,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.952971e-12
epoch=468,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.755323e-12
epoch=469,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.5675566e-12
epoch=470,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.389179e-12
epoch=471,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.2197198e-12
epoch=472,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.0587338e-12
epoch=473,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.9057971e-12
epoch=474,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.7605073e-12
epoch=475,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6224819e-12
epoch=476,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4913578e-12
epoch=477,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.36679e-12
epoch=478,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2484506e-12
epoch=479,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.136028e-12
epoch=480,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0292266e-12
epoch=481,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9277652e-12
epoch=482,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.831377e-12
epoch=483,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7398082e-12
epoch=484,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6528177e-12
epoch=485,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5701768e-12
epoch=486,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.491668e-12
epoch=487,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4170845e-12
epoch=488,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3462303e-12
epoch=489,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2789188e-12
epoch=490,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2149729e-12
epoch=491,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1542243e-12
epoch=492,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0965131e-12
epoch=493,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0416874e-12
epoch=494,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.89603e-13
epoch=495,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.401229e-13
epoch=496,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.9311674e-13
epoch=497,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.484609e-13
epoch=498,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.0603785e-13
epoch=499,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.6573594e-13
epoch=500,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.2744913e-13
epoch=501,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.910767e-13
epoch=502,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.5652285e-13
epoch=503,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.2369673e-13
epoch=504,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.925119e-13
epoch=505,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.6288627e-13
epoch=506,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.3474196e-13
epoch=507,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.080049e-13
epoch=508,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.826046e-13
epoch=509,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.584744e-13
epoch=510,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.3555068e-13
epoch=511,test accuracy=0.03125,train accuracy=0.03125,learning rate=4.1377316e-13
epoch=512,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.930845e-13
epoch=513,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.7343026e-13
epoch=514,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.5475876e-13
epoch=515,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.370208e-13
epoch=516,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.2016978e-13
epoch=517,test accuracy=0.03125,train accuracy=0.03125,learning rate=3.041613e-13
epoch=518,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.8895321e-13
epoch=519,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.7450557e-13
epoch=520,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.6078028e-13
epoch=521,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.4774128e-13
epoch=522,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.3535422e-13
epoch=523,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.2358649e-13
epoch=524,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.1240717e-13
epoch=525,test accuracy=0.03125,train accuracy=0.03125,learning rate=2.0178682e-13
epoch=526,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.9169747e-13
epoch=527,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.821126e-13
epoch=528,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.7300697e-13
epoch=529,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.6435662e-13
epoch=530,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.5613879e-13
epoch=531,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4833185e-13
epoch=532,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.4091526e-13
epoch=533,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.3386949e-13
epoch=534,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2717602e-13
epoch=535,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.2081722e-13
epoch=536,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.1477635e-13
epoch=537,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.0903754e-13
epoch=538,test accuracy=0.03125,train accuracy=0.03125,learning rate=1.03585665e-13
epoch=539,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.840638e-14
epoch=540,test accuracy=0.03125,train accuracy=0.03125,learning rate=9.348606e-14
epoch=541,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.881176e-14
epoch=542,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.437117e-14
epoch=543,test accuracy=0.03125,train accuracy=0.03125,learning rate=8.0152614e-14
epoch=544,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.614498e-14
epoch=545,test accuracy=0.03125,train accuracy=0.03125,learning rate=7.233773e-14
epoch=546,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.872085e-14
epoch=547,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.52848e-14
epoch=548,test accuracy=0.03125,train accuracy=0.03125,learning rate=6.202056e-14
epoch=549,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.891954e-14
epoch=550,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.5973557e-14
epoch=551,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.317488e-14
epoch=552,test accuracy=0.03125,train accuracy=0.03125,learning rate=5.0516137e-14